# Portfolio Optimization Tool

# Investor Options

## 1. Market Selection (WIP)

In [ ]:
MARKET_SELECTIONS = Dict{String, Bool}(
    "SP500" => true,
    "DOW" => false,
    "NASDAQ" => false,
    "CRYPTO" => true,
    "COMMOD" => false,
    "FOREX" => false,
    "BONDS" => false,
    "MUTUAL" => false,
    "CUSTOM" => false
);

## 2. Diversity

In [ ]:
SECURITY_ALLOCATION_LIMIT = 0.2;
INDUSTRY_ALLOCATION_LIMIT = 0.2;
MARKET_ALLOCATION_LIMIT = 0.2;

## 3. Alternate Currency (WIP)

In [ ]:
ALTERNATE_CURRENCY = "USD";

## 4. Risk Tolerance (WIP)

In [ ]:
RISK_ROLERANCE = Dict{String, Bool}(
    "LOW" => false,
    "MEDIUM" => false,
    "HIGH" => false,
    "BENTLEYS_OR_BRIDGES" => true
);

## 5. Date Range (WIP)

In [ ]:
using Dates

startDate = "2005-01-01";
endDate = "2021-01-01";

START_DATE = datetime2unix(DateTime(startDate,"yyyy-mm-dd"));
END_DATE = datetime2unix(DateTime(endDate,"yyyy-mm-dd"));

# Code

## 1. Import Packages

In [ ]:
import Dates
using Pkg
Pkg.add("Latexify");
Pkg.add("LaTeXStrings");
Pkg.add("LinearAlgebra");
Pkg.add("JuMP");
Pkg.add("PyPlot");
Pkg.add("NamedArrays");
Pkg.add("CSV");
Pkg.add("DataFrames");
Pkg.add("Distributions");
Pkg.add("Ipopt");
Pkg.add("HTTP");
Pkg.add("JSON");
;

## 2. Write Generic API Clients

In [ ]:
using HTTP
using JSON
using DataFrames

# Write some general API client request functions

# Uses the TIME_SERIES_DAILY_ADJUSTED api endpoint
# https://www.alphavantage.co/documentation/#dailyadj
function retrieveStockTimeSeriesData(ticker)
    
    apikey = "N5ERQO6K97VZX2Z8"
    base = "https://www.alphavantage.co/query?"
    
    url = string(
        base, 
        "function=TIME_SERIES_DAILY_ADJUSTED",
        "&symbol=", ticker,
        "&datatype=json",
        "&apikey=", apikey,
        "&outputsize=full"
    )
    
    response = HTTP.get(url);
    response_text = String(response.body);
    json_obj = JSON.parse(response_text);
    return (json_obj);
end

# Used the DIGITAL_CURRENCY_DAILY api endpoint
# https://www.alphavantage.co/documentation/#currency-daily
function retrieveCryptocurrencySecurityTimeSeriesData(symbol, market)
    
    apikey = "N5ERQO6K97VZX2Z8"
    base = "https://www.alphavantage.co/query?"
    
    url = string(
        base, 
        "function=DIGITAL_CURRENCY_DAILY",
        "&symbol=", symbol,
        "&apikey=", apikey,
        "&market=", market,
        "&datatype=json",
        "&outputsize=full"
    )
    
    response = HTTP.get(url)
    response_text = String(response.body)
    json_obj = JSON.parse(response_text)
    return (json_obj);
end
;

## 3. Define Structs

In [ ]:
# Define the structs used in data management / organization

mutable struct marketObject
    marketId # - Int64
    marketName # - String
    numberIndustries # - Int64
    industryObjectKeys # - Vector{String(industryName)}
    industryObjects # - Dict : String(industryName) => industryObject
    numberSecurities # - Int64
    securityObjectKeys # - Vector{String(tickerSymbol)}
    securityObjects # - Dict : String(tickerSymbol) => securityObject
end

mutable struct industryObject
    industryId # - Int64
    industryName # - String
    numberSecurities # - Int64
    securityObjectKeys # - Vector{String(tickerSymbol)}
    securityObjects # - Dict : String(tickerSymbol) => securityObject
end

mutable struct securityObject
    tickerId # - Int64
    tickerSymbol # - String
    securityName # - String
    market # - String
    industry # - String
    sortedDailyPerformanceObjectKeys # - Vector{Float64(numericalDateKey)}
    dailyPerformanceObjects # - Dict : Float64(numericalDateKey) => dailyPerformanceObject
end

mutable struct dailyPerformanceObject
    date # - Float64
    openUSD # - Float64
    closeUSD # - Float64
    adjustedCloseUSD # - Float64
    highUSD # - Float64
    lowUSD # - Float64
    openAlt # - Float64
    closeAlt # - Float64
    highAlt # - Float64
    lowAlt # - Float64
    volume # - Float64
    dividendAmount # - Float64
    splitCoefficient # - Float64
    marketCap # - Float64
    previousPercentChange # - Float64: last n-1 days, String: if first day
end

## 4. Declare Variables

In [ ]:
using CSV, DataFrames

# Project Seed Data Retrieval / Setup

tickers = []; # - Vector{String}
names = []; # - Vector{String}
industries = []; # - Vector{String}
markets = []; # - Vector{String}

if (MARKET_SELECTIONS["SP500"])
    sp500 = CSV.read("SP500Tickers.csv", header=0, DataFrame); # - DataFrame
    tickers = [tickers; sp500[:, 1]];
    names = [names; sp500[:, 2]];
    industries = [industries; sp500[:, 3]];
    markets = [markets; fill("SP500", (length(sp500[:, 1]),1))]
end

if (MARKET_SELECTIONS["DOW"])
    dow = CSV.read("DOWTickers.csv", header=0, DataFrame); # - DataFrame
    tickers = [tickers; dow[:, 1]];
    names = [names; dow[:, 2]];
    industries = [industries; dow[:, 3]];
    markets = [markets; fill("DOW", (length(dow[:, 1]),1))]
end

if (MARKET_SELECTIONS["NASDAQ"])
    nasdaq = CSV.read("NASDAQTickers.csv", header=0, DataFrame); # - DataFrame
    tickers = [tickers; nasdaq[:, 1]];
    names = [names; nasdaq[:, 2]];
    industries = [industries; nasdaq[:, 3]];
    markets = [markets; fill("NASDAQ", (length(nasdaq[:, 1]),1))]
end

if (MARKET_SELECTIONS["CRYPTO"])
    cryptocurrencies = CSV.read("CryptocurrencyTickers.csv", header=0, DataFrame); # - DataFrame
    tickers = [tickers; cryptocurrencies[:, 1]];
    names = [names; cryptocurrencies[:, 2]];
    industries = [industries; cryptocurrencies[:, 3]];
    markets = [markets; fill("CRYPTO", (length(cryptocurrencies[:, 1]),1))]
end

if (MARKET_SELECTIONS["COMMOD"])
    commodities = CSV.read("CommodityTickers.csv", header=0, DataFrame); # - DataFrame
    tickers = [tickers; commodities[:, 1]];
    names = [names; commodities[:, 2]];
    industries = [industries; commodities[:, 3]];
    markets = [markets; fill("COMMOD", (length(commodities[:, 1]),1))]
end

if (MARKET_SELECTIONS["FOREX"])
    forex = CSV.read("FOREXTickers.csv", header=0, DataFrame); # - DataFrame
    tickers = [tickers; forex[:, 1]];
    names = [names; forex[:, 2]];
    industries = [industries; forex[:, 3]];
    markets = [markets; fill("FOREX", (length(forex[:, 1]),1))]
end

if (MARKET_SELECTIONS["BONDS"])
    bonds = CSV.read("GovernmentBondTickers.csv", header=0, DataFrame); # - DataFrame
    tickers = [tickers; bonds[:, 1]];
    names = [names; bonds[:, 2]];
    industries = [industries; bonds[:, 3]];
    markets = [markets; fill("BONDS", (length(bonds[:, 1]),1))]
end

if (MARKET_SELECTIONS["MUTUAL"])
    mutualFunds = CSV.read("MutualFundTickers.csv", header=0, DataFrame); # - DataFrame
    tickers = [tickers; mutualFunds[:, 1]];
    names = [names; mutualFunds[:, 2]];
    industries = [industries; mutualFunds[:, 3]];
    markets = [markets; fill("MUTUAL", (length(mutualFunds[:, 1]),1))]
end

if (MARKET_SELECTIONS["CUSTOM"])
    customSecurities = CSV.read("__FILLIN__.csv", header=0, DataFrame); # - DataFrame
    tickers = [tickers; customSecurities[:, 1]];
    names = [names; customSecurities[:, 2]];
    industries = [industries; customSecurities[:, 3]];
    markets = [markets; fill("CUSTOM", (length(customSecurities[:, 1]),1))]
end

numberSecurities = length(tickers); # - Int64

# Market Data Structures
marketIdMap = Dict{Int64, String}();
marketHistoricalReturns = Dict{String, Float64}();
marketHistories = Dict{String, marketObject}();

# Industry Data Structures
industryIdMap = Dict{Int64, String}();
industryHistoricalReturns = Dict{String, Float64}();
industryHistories = Dict{String, industryObject}();

# Securities Data Structures
securityIdMap = Dict{Int64, String}();
securityHistoricalReturns = Dict{String, Float64}();
securityHistories = Dict{String, securityObject}();

## 5. Data Retrieval

In [ ]:
using Dates

marketIdCounter = 0; # - Int64
industryIdCounter = 0; # - Int64
securityIdCounter = 0; # - Int64

for i in 1:numberSecurities

    curSecurityMarket = markets[i]
    # Check if this is a brand new market, if so, create a new market object
    if !haskey(marketHistories, curSecurityMarket)
        # Dict does not have this industry, create a new entry
        marketIdCounter += 1;
        newIndustryObjectKeysVector = Vector{String}();
        newIndustryObjectsDict = Dict{String, industryObject}();
        newSecurityObjectKeysVector = Vector{String}();
        newSecurityObjectsDict = Dict{String, securityObject}();
        
        newMarketObject = marketObject(
            marketIdCounter,
            curSecurityMarket,
            0,
            newIndustryObjectKeysVector,
            newIndustryObjectsDict,
            0,
            newSecurityObjectKeysVector,
            newSecurityObjectsDict
        );
        
        marketHistories[curSecurityMarket] = newMarketObject
        marketHistoricalReturns[curSecurityMarket] = 0
        marketIdMap[marketIdCounter] = curSecurityMarket;
    end
    
    curSecurityIndustry = industries[i]
    # Check if this is a brand new industry, if so, create a new industry object
    if !haskey(industryHistories, curSecurityIndustry)
        # Dict does not have this industry, create a new entry
        industryIdCounter += 1;
        newSecurityObjectKeysVector = Vector{String}();
        newSecurityObjectsDict = Dict{String, securityObject}();
        
        newIndustryObject = industryObject(
            industryIdCounter,
            curSecurityIndustry,
            0,
            newSecurityObjectKeysVector,
            newSecurityObjectsDict
        );
        
        industryHistories[curSecurityIndustry] = newIndustryObject
        industryHistoricalReturns[curSecurityIndustry] = 0
        industryIdMap[industryIdCounter] = curSecurityIndustry;
    end
    
    securityIdCounter += 1;
    curSecurityTicker = tickers[i]
    curSecurityName = names[i]
    
    if (curSecurityMarket == "SP500" || curSecurityMarket == "DOW" || curSecurityMarket == "NASDAQ")
        PARSING_MODE = 1
        
        TIME_SERIES_OUTER_KEY = "Time Series (Daily)";
        DAILY_OPEN_KEY = "1. open";
        DAILY_CLOSE_KEY = "4. close";
        DAILY_HIGH_KEY = "2. high";
        DAILY_LOW_KEY = "3. low";
        DAILY_ADJUSTED_CLOSE_KEY = "5. adjusted close";
        DAILY_VOLUME_KEY = "6. volume";
        DAILY_DIVIDEND_AMOUNT_KEY = "7. dividend amount";
        DAILY_SPLIT_COEFFICIENT_KEY = "8. split coefficient";
        
        curSecurityData = retrieveStockTimeSeriesData(curSecurityTicker);
    elseif (curSecurityMarket == "CRYPTO")
        PARSING_MODE = 2
        
        TIME_SERIES_OUTER_KEY = "Time Series (Digital Currency Daily)";
        DAILY_OPEN_KEY = "1b. open (USD)";
        DAILY_CLOSE_KEY = "4b. close (USD)";
        DAILY_HIGH_KEY = "2b. high (USD)";
        DAILY_LOW_KEY = "3b. low (USD)";
        DAILY_ALT_OPEN_KEY = string("1a. open (",ALTERNATE_CURRENCY,")");
        DAILY_ALT_CLOSE_KEY = string("4a. close (",ALTERNATE_CURRENCY,")");
        DAILY_ALT_HIGH_KEY = string("2a. high (",ALTERNATE_CURRENCY,")");
        DAILY_ALT_LOW_KEY = string("3a. low (",ALTERNATE_CURRENCY,")");
        DAILY_VOLUME_KEY = "5. volume";
        DAILY_MARKETCAP_KEY = "6. market cap (USD)";
        
        curSecurityData = retrieveCryptocurrencySecurityTimeSeriesData(curSecurityTicker, ALTERNATE_CURRENCY);
    elseif (curSecurityMarket == "COMMOD")
        PARSING_MODE = 3
        TIME_SERIES_OUTER_KEY = "";
        curSecurityData = retrieveCommoditySecurityTimeSeriesData();
    elseif (curSecurityMarket == "FOREX")
        PARSING_MODE = 4
        TIME_SERIES_OUTER_KEY = "";
        curSecurityData = retrieveFOREXSecurityTimeSeriesData();
    elseif (curSecurityMarket == "BONDS")
        PARSING_MODE = 5
        TIME_SERIES_OUTER_KEY = "";
        curSecurityData = retrieveGovernmentBondSecurityTimeSeriesData();
    elseif (curSecurityMarket == "MUTUAL")
        PARSING_MODE = 6
        TIME_SERIES_OUTER_KEY = "";
        curSecurityData = retrieveMutualFundSecurityTimeSeriesData();
    else
        # UHHHH, How did we get here?
        println("ERROR IN MARKET PARSING")
        println("===============================")
        print("Index: ")
        println(i)
        print("Ticker: ")
        println(curSecurityTicker)
        print("Market: ")
        println(curSecurityMarket)
        print("Industry: ")
        println(curSecurityIndustry)
        println()
    end
    
    # Retrieve the array fo time series data points
    curSecurityDailyTimeSeries = curSecurityData[TIME_SERIES_OUTER_KEY]
    numberDataPoints = length(curSecurityDailyTimeSeries)
    
    # For temporary storage
    tempDataDict = Dict{Float64, Any}()
    
    # YYYY-MM-DD - need to quantify
    dates = collect(keys(curSecurityDailyTimeSeries))
    # For sorting
    date_times = zeros(length(dates));
    
    for j in 1:length(dates)
        # Create a Date Object from the key
        curDateObject = DateTime(dates[j],"yyyy-mm-dd")
        # Get the epoch time
        curDateTime = datetime2unix(curDateObject) # Float64
        # Add the entry to our date_times vector
        date_times[j] = curDateTime;
        # Get the data, Dict(String, Any)
        curDateData = curSecurityDailyTimeSeries[dates[j]];
        # Set the new quantified key and data in other Dict
        tempDataDict[curDateTime] = curDateData;
    end
    
    # Now we need to sort the date_times, Vector{Float64}
    SortedNumericalKeys = sort(date_times);
    
    # Put the data into new dict, Dict : numericalDateKey => priceHistoryDataPointObject
    securityDailyHistory = Dict{Float64, dailyPerformanceObject}();
    
    previousPercentChangeSum = 0;
    previousDayClose = 0;
    
    for j in 1:length(SortedNumericalKeys)
        curKey = SortedNumericalKeys[j]; # Float64
        curKeyData = tempDataDict[curKey];
        
        newPerformanceObjectData_date = curKey;
        newPerformanceObjectData_openUSD = parse(Float64,curKeyData[DAILY_OPEN_KEY]);
        newPerformanceObjectData_closeUSD = parse(Float64,curKeyData[DAILY_CLOSE_KEY]);
        newPerformanceObjectData_highUSD = parse(Float64,curKeyData[DAILY_HIGH_KEY]);
        newPerformanceObjectData_lowUSD = parse(Float64,curKeyData[DAILY_LOW_KEY]);
        newPerformanceObjectData_adjustedCloseUSD = -102030201.00;
        newPerformanceObjectData_openAlt = -102030201.00;
        newPerformanceObjectData_closeAlt = -102030201.00;
        newPerformanceObjectData_highAlt = -102030201.00;
        newPerformanceObjectData_lowAlt = -102030201.00;
        newPerformanceObjectData_volume = parse(Float64,curKeyData[DAILY_VOLUME_KEY]);
        newPerformanceObjectData_dividendAmount = -102030201.00;
        newPerformanceObjectData_splitCoefficient = -102030201.00;
        newPerformanceObjectData_marketCap = -102030201;
        
        newPerformanceObjectData_previousPercentChange = -102030201.00;
        
        if (PARSING_MODE == 1)
            newPerformanceObjectData_adjustedCloseUSD = parse(Float64,curKeyData[DAILY_ADJUSTED_CLOSE_KEY]);
            newPerformanceObjectData_dividendAmount = parse(Float64,curKeyData[DAILY_DIVIDEND_AMOUNT_KEY]);
            newPerformanceObjectData_splitCoefficient = parse(Float64,curKeyData[DAILY_SPLIT_COEFFICIENT_KEY]);
        elseif (PARSING_MODE == 2)
            newPerformanceObjectData_openAlt = parse(Float64,curKeyData[DAILY_ALT_OPEN_KEY]);
            newPerformanceObjectData_closeAlt = parse(Float64,curKeyData[DAILY_ALT_CLOSE_KEY]);
            newPerformanceObjectData_highAlt = parse(Float64,curKeyData[DAILY_ALT_HIGH_KEY]);
            newPerformanceObjectData_lowAlt = parse(Float64,curKeyData[DAILY_ALT_LOW_KEY]);
            newPerformanceObjectData_marketCap = parse(Float64,curKeyData[DAILY_MARKETCAP_KEY]);
        elseif (PARSING_MODE == 3)
            
        elseif (PARSING_MODE == 4)
            
        elseif (PARSING_MODE == 5)
            
        elseif (PARSING_MODE == 6)
            
        else
            # UHHHH, How did we get here?
            println("ERROR IN PARSING_MODE PARSING")
            println("===============================")
            print("Index: ")
            println(i)
            print("Ticker: ")
            println(curSecurityTicker)
            print("Market: ")
            println(curSecurityMarket)
            print("Industry: ")
            println(curSecurityIndustry)
            println()
        end
        
        # Calculate % change from close to close
        if (j == 1)
            newPerformanceObjectData_previousPercentChange = 
                (newPerformanceObjectData_closeUSD - newPerformanceObjectData_openUSD) / newPerformanceObjectData_openUSD
            
            previousPercentChangeSum += newPerformanceObjectData_previousPercentChange
            industryHistoricalReturns[curSecurityIndustry] += newPerformanceObjectData_previousPercentChange
            marketHistoricalReturns[curSecurityMarket] += newPerformanceObjectData_previousPercentChange
            previousDayClose = newPerformanceObjectData_closeUSD
        else
            newPerformanceObjectData_previousPercentChange = 
                (newPerformanceObjectData_closeUSD - previousDayClose) / previousDayClose
            
            previousPercentChangeSum += newPerformanceObjectData_previousPercentChange
            industryHistoricalReturns[curSecurityIndustry] += newPerformanceObjectData_previousPercentChange
            marketHistoricalReturns[curSecurityMarket] += newPerformanceObjectData_previousPercentChange
            previousDayClose = newPerformanceObjectData_closeUSD
        end
        
        # Create the cryptocurrencyPriceHistoryDataPoint object for representation
        newDailyPerformanceObject = dailyPerformanceObject(
            newPerformanceObjectData_date,
            newPerformanceObjectData_openUSD,
            newPerformanceObjectData_closeUSD,
            newPerformanceObjectData_adjustedCloseUSD,
            newPerformanceObjectData_highUSD,
            newPerformanceObjectData_lowUSD,
            newPerformanceObjectData_openAlt,
            newPerformanceObjectData_closeAlt,
            newPerformanceObjectData_highAlt,
            newPerformanceObjectData_lowAlt,
            newPerformanceObjectData_volume,
            newPerformanceObjectData_dividendAmount,
            newPerformanceObjectData_splitCoefficient,
            newPerformanceObjectData_marketCap,
            newPerformanceObjectData_previousPercentChange
        );
        
        securityDailyHistory[curKey] = newDailyPerformanceObject
    end
    
    securityHistoricalReturns[curSecurityTicker] = previousPercentChangeSum / length(SortedNumericalKeys)
    
    # Create the stockTickerObject object for representation
    newSecurityObject = securityObject(
        securityIdCounter,
        curSecurityTicker,
        curSecurityName,
        curSecurityMarket,
        curSecurityIndustry,
        SortedNumericalKeys,
        securityDailyHistory
    );
    
    # Add to securityHistories - Dict{String(Ticker) => securityObject}
    securityHistories[curSecurityTicker] = newSecurityObject;
    securityIdMap[securityIdCounter] = curSecurityTicker;
    
    # Add the securityObject into the relevant industry data structures
    industryHistories[curSecurityIndustry].numberSecurities += 1
    industryHistories[curSecurityIndustry].securityObjects[curSecurityTicker] = newSecurityObject
    push!(industryHistories[curSecurityIndustry].securityObjectKeys, curSecurityTicker)
    
    # Add the securityObject into the relevant market data structures
    marketHistories[curSecurityMarket].numberSecurities += 1
    marketHistories[curSecurityMarket].securityObjects[curSecurityTicker] = newSecurityObject
    push!(marketHistories[curSecurityMarket].securityObjectKeys, curSecurityTicker)
end



In [ ]:
# Convert the total sums into averages
industryKeys = collect(keys(industryHistories));

for i in 1:length(industryKeys)
    curIndustryKey = industryKeys[i];
    curIndustryObject = industryHistories[curIndustryKey];
    
    # Get the raw sum
    curIndustryReturnSum = industryHistoricalReturns[curIndustryKey];
    
    # Need to find the number of days used to calculate this sum
    curIndustrySecurityTickers = curIndustryObject.securityObjectKeys
    curIndustrySecurityDict = curIndustryObject.securityObjects
    
    numberDays = 0;
    numberIndustrySecurities = length(curIndustrySecurityTickers)
    
    for j in 1:numberIndustrySecurities
        curSecurityKey = curIndustrySecurityTickers[j]
        curSecurityObject =  curIndustrySecurityDict[curSecurityKey]
        
        numberDays += length(curSecurityObject.sortedDailyPerformanceObjectKeys)
    end
    
    # Now we have the number of days used in the sums, find and set averages
    industryHistoricalReturns[curIndustryKey] = curIndustryReturnSum / numberDays; 
end

marketKeys = collect(keys(marketHistories));

for i in 1:length(marketKeys)
    curMarketKey = marketKeys[i];
    curMarketObject = marketHistories[curMarketKey];
    
    curMarketSecurityTickers = curMarketObject.securityObjectKeys
    curMarketSecurityDict = curMarketObject.securityObjects
    
    # Get the raw sum
    curMarketReturnSum = marketHistoricalReturns[curMarketKey];
    
    numberDays = 0;
    numberMarketSecurities = length(curMarketSecurityTickers)
    
    for j in 1:numberMarketSecurities
        curSecurityKey = curMarketSecurityTickers[j]
        curSecurityObject = curMarketSecurityDict[curSecurityKey]
        
        curSecurityIndustryKey = curSecurityObject.industry
        
        if !haskey(curMarketObject.industryObjects, curSecurityIndustryKey)
            curMarketObject.industryObjects[curSecurityIndustryKey] = industryHistories[curSecurityIndustryKey]
            push!(marketHistories[curMarketKey].industryObjectKeys, curSecurityIndustryKey)
        end
        
        numberDays += length(curSecurityObject.sortedDailyPerformanceObjectKeys)
    end
    
    marketHistoricalReturns[curMarketKey] = curMarketReturnSum / numberDays
end


## 6. Compute Covariance Matrices

### 6.A. Securities

In [ ]:
securityKeys = collect(keys(securityHistories))
numberSecurities = length(securityKeys)
SecurityCovarianceMatrix = zeros(Float64, numberSecurities, numberSecurities)

for i in 1:numberSecurities
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId
    
    curSecurityPriceHistorySortedKeys = curSecurityObject.sortedDailyPerformanceObjectKeys
    curSecurityPriceHistory = curSecurityObject.dailyPerformanceObjects
    
    curSecurityHistoricalReturn = securityHistoricalReturns[curSecurityKey]
    
    for j in i:numberSecurities
        
        if (i == j) # Calculate the variance within this security's time series data
            varianceSum = 0;
            
            for k in 1:length(curSecurityPriceHistorySortedKeys)
                curDateKey = curSecurityPriceHistorySortedKeys[k]
                curPerformanceObject = curSecurityPriceHistory[curDateKey]
                
                curPerformanceObjectPreviousPercentChange = curPerformanceObject.previousPercentChange
                varianceSum += (curPerformanceObjectPreviousPercentChange - curSecurityHistoricalReturn)^2
            end
            
            curSecurityVariance = varianceSum / (length(curSecurityPriceHistorySortedKeys) - 1)
            SecurityCovarianceMatrix[curSecurityObjectId, curSecurityObjectId] = curSecurityVariance
        else # Calculate the covariance between both security's time series data
            altSecurityKey = securityKeys[j]
            altSecurityObject = securityHistories[altSecurityKey]
            altSecurityObjectId = altSecurityObject.tickerId
            
            altSecurityPriceHistorySortedKeys = altSecurityObject.sortedDailyPerformanceObjectKeys
            altSecurityPriceHistory = altSecurityObject.dailyPerformanceObjects
            
            altSecurityHistoricalReturn = securityHistoricalReturns[altSecurityKey]
            
            covarianceSum = 0;
            skippedDays = 0;
            
            if (length(curSecurityPriceHistorySortedKeys) > length(altSecurityPriceHistorySortedKeys))
                
                for k in 1:length(altSecurityPriceHistorySortedKeys)
                    curDateKey = altSecurityPriceHistorySortedKeys[k]
                    
#                     if haskey(curSecurityPriceHistorySortedKeys, curDateKey)
                    if (curDateKey in curSecurityPriceHistorySortedKeys)
                        curSecurityPriceHistoryObject = curSecurityPriceHistory[curDateKey]
                        altSecurityPriceHistoryObject = altSecurityPriceHistory[curDateKey]
                        
                        curSecurityPreviousPercentChange = curSecurityPriceHistoryObject.previousPercentChange
                        altSecurityPreviousPercentChange = altSecurityPriceHistoryObject.previousPercentChange
                        
                        curSecurityDelta = curSecurityPreviousPercentChange - curSecurityHistoricalReturn
                        altSecurityDelta = altSecurityPreviousPercentChange - altSecurityHistoricalReturn
                        
                        covarianceSum += curSecurityDelta * altSecurityDelta
                    else
                       skippedDays += 1; 
                    end
                end
                
                covariance = covarianceSum / (length(altSecurityPriceHistorySortedKeys) - (1 + skippedDays))
                SecurityCovarianceMatrix[curSecurityObjectId, altSecurityObjectId] = covariance
                SecurityCovarianceMatrix[altSecurityObjectId, curSecurityObjectId] = covariance
            else
                
                for k in 1:length(curSecurityPriceHistorySortedKeys)
                    curDateKey = curSecurityPriceHistorySortedKeys[k]
                    
#                     if haskey(altSecurityPriceHistorySortedKeys, curDateKey)
                    if (curDateKey in altSecurityPriceHistorySortedKeys)
                        curSecurityPriceHistoryObject = curSecurityPriceHistory[curDateKey]
                        altSecurityPriceHistoryObject = altSecurityPriceHistory[curDateKey]
                        
                        curSecurityPreviousPercentChange = curSecurityPriceHistoryObject.previousPercentChange
                        altSecurityPreviousPercentChange = altSecurityPriceHistoryObject.previousPercentChange
                        
                        curSecurityDelta = curSecurityPreviousPercentChange - curSecurityHistoricalReturn
                        altSecurityDelta = altSecurityPreviousPercentChange - altSecurityHistoricalReturn
                        
                        covarianceSum += curSecurityDelta * altSecurityDelta
                    else
                       skippedDays += 1; 
                    end
                end
                
                covariance = covarianceSum / (length(curSecurityPriceHistorySortedKeys) - (1 + skippedDays))
                SecurityCovarianceMatrix[curSecurityObjectId, altSecurityObjectId] = covariance
                SecurityCovarianceMatrix[altSecurityObjectId, curSecurityObjectId] = covariance
            end 
        end 
    end
end

### 6.B. Industries

In [ ]:
industryKeys = collect(keys(industryHistories))
numberIndustries = length(industryKeys)
IndustryCovarianceMatrix = zeros(Float64, numberIndustries, numberIndustries)

combinedSecuritySortedDateKeyDict = Dict{String, Any}();

for i in 1:numberIndustries
    curIndustryKey = industryKeys[i]
    curIndustryObject = industryHistories[curIndustryKey]
    curIndustryObjectId = curIndustryObject.industryId
    
    curIndustrySecurityObjectKeys = curIndustryObject.securityObjectKeys
    curIndustrySecurityObjects = curIndustryObject.securityObjects
    
    curIndustryHistoricalReturn = industryHistoricalReturns[curIndustryKey]
    
    if !haskey(combinedSecuritySortedDateKeyDict, curIndustryKey)
        uniqueKeyDict = Dict{Float64, Any}();
        
        for j in 1:length(curIndustrySecurityObjectKeys)
            curSecurityObjectKey = curIndustrySecurityObjectKeys[j]
            curSecurityObject = curIndustrySecurityObjects[curSecurityObjectKey]
            
            for k in 1:length(curSecurityObject.sortedDailyPerformanceObjectKeys)
                curDateKey = curSecurityObject.sortedDailyPerformanceObjectKeys[k]
                
                if haskey(uniqueKeyDict, curDateKey)
                    uniqueKeyDict[curDateKey][curSecurityObjectKey] = curSecurityObject.dailyPerformanceObjects[curDateKey]
                else
                    newDayDict = Dict{String, dailyPerformanceObject}();
                    newDayDict[curSecurityObjectKey] = curSecurityObject.dailyPerformanceObjects[curDateKey]
                    
                    uniqueKeyDict[curDateKey] = newDayDict;
                end
            end
        end
        
        combinedSecuritySortedDateKeyDict[curIndustryKey] = uniqueKeyDict
    end
    
    for j in i:numberIndustries
        
        if (i == j) # Calculate the average variance across this industry's securities
            varianceSum = 0;
            curDateKeySet = sort(collect(keys(combinedSecuritySortedDateKeyDict[curIndustryKey])))
            
            for k in 1:length(curDateKeySet)
                curDateKey = curDateKeySet[k]
                curDateSecurityObjectKeys = collect(keys(combinedSecuritySortedDateKeyDict[curIndustryKey][curDateKey]))
                
                returnSum = 0;
                
                for m in 1:length(curDateSecurityObjectKeys)
                    curSecurityObjectKey = curDateSecurityObjectKeys[m]
                    curSecurityPerformanceObject = combinedSecuritySortedDateKeyDict[curIndustryKey][curDateKey][curSecurityObjectKey]
                
                    curSecurityPreviousPercentChange = curSecurityPerformanceObject.previousPercentChange
                
                    returnSum += curSecurityPreviousPercentChange
                end
                
                varianceSum += ((returnSum / length(curDateSecurityObjectKeys)) - curIndustryHistoricalReturn)^2
            end
            
            variance = varianceSum / (length(curDateKeySet) - 1)
            IndustryCovarianceMatrix[curIndustryObjectId, curIndustryObjectId] = variance
        else # Calculate the covariance
            altIndustryKey = industryKeys[j]
            altIndustryObject = industryHistories[altIndustryKey]
            altIndustryObjectId = altIndustryObject.industryId

            altIndustrySecurityObjectKeys = altIndustryObject.securityObjectKeys
            altIndustrySecurityObjects = altIndustryObject.securityObjects

            altIndustryHistoricalReturn = industryHistoricalReturns[altIndustryKey]
        
            
            if !haskey(combinedSecuritySortedDateKeyDict, altIndustryKey)
                uniqueKeyDict = Dict{Float64, Any}();

                for k in 1:length(altIndustrySecurityObjectKeys)
                    altSecurityObjectKey = altIndustrySecurityObjectKeys[k]
                    altSecurityObject = altIndustrySecurityObjects[altSecurityObjectKey]

                    for m in 1:length(altSecurityObject.sortedDailyPerformanceObjectKeys)
                        altDateKey = altSecurityObject.sortedDailyPerformanceObjectKeys[m]

                        if haskey(uniqueKeyDict, altDateKey)
                            uniqueKeyDict[altDateKey][altSecurityObjectKey] = altSecurityObject.dailyPerformanceObjects[altDateKey]
                        else
                            newDayDict = Dict{String, dailyPerformanceObject}();
                            newDayDict[altSecurityObjectKey] = altSecurityObject.dailyPerformanceObjects[altDateKey]

                            uniqueKeyDict[altDateKey] = newDayDict;
                        end
                    end
                end

                combinedSecuritySortedDateKeyDict[altIndustryKey] = uniqueKeyDict
            end
            
            curIndustryDateKeySet = sort(collect(keys(combinedSecuritySortedDateKeyDict[curIndustryKey])))
            altIndustryDateKeySet = sort(collect(keys(combinedSecuritySortedDateKeyDict[altIndustryKey])))
        
            if (length(curIndustryDateKeySet) > length(altIndustryDateKeySet))
                covarianceSum = 0;
                skippedDays = 0;
                
                for k in 1:length(altIndustryDateKeySet)
                    curDateKey = altIndustryDateKeySet[k]
                    
                    if curDateKey in curIndustryDateKeySet
                        
                        curIndustrySum = 0;
                        curIndustrySecurities = collect(keys(combinedSecuritySortedDateKeyDict[curIndustryKey][curDateKey]))
                        for m in 1:length(curIndustrySecurities)
                            curSecurityObjectKey = curIndustrySecurities[m]
                            curSecurityObject = combinedSecuritySortedDateKeyDict[curIndustryKey][curDateKey][curSecurityObjectKey]
                            
                            curIndustrySum += curSecurityObject.previousPercentChange;
                        end
                        
                        altIndustrySum = 0;
                        altIndustrySecurities = collect(keys(combinedSecuritySortedDateKeyDict[altIndustryKey][curDateKey]))
                        for m in 1:length(altIndustrySecurities)
                            altSecurityObjectKey = altIndustrySecurities[m]
                            altSecurityObject = combinedSecuritySortedDateKeyDict[altIndustryKey][curDateKey][altSecurityObjectKey]
                            
                            altIndustrySum += altSecurityObject.previousPercentChange;
                        end
                        
                        covarianceSum += ((curIndustrySum / length(curIndustrySecurities)) - curIndustryHistoricalReturn) * ((altIndustrySum / length(altIndustrySecurities)) - altIndustryHistoricalReturn)
                    else
                        skippedDays += 1
                    end
                    
                    covariance = covarianceSum / (length(altIndustryDateKeySet) - (1 + skippedDays))
                    IndustryCovarianceMatrix[curIndustryObjectId,altIndustryObjectId] = covariance;
                    IndustryCovarianceMatrix[altIndustryObjectId,curIndustryObjectId] = covariance;
                end
            else
                covarianceSum = 0;
                skippedDays = 0;
                
                for k in 1:length(curIndustryDateKeySet)
                    curDateKey = curIndustryDateKeySet[k]
                    
                    if curDateKey in altIndustryDateKeySet
                        
                        curIndustrySum = 0;
                        curIndustrySecurities = collect(keys(combinedSecuritySortedDateKeyDict[curIndustryKey][curDateKey]))
                        for m in 1:length(curIndustrySecurities)
                            curSecurityObjectKey = curIndustrySecurities[m]
                            curSecurityObject = combinedSecuritySortedDateKeyDict[curIndustryKey][curDateKey][curSecurityObjectKey]
                            
                            curIndustrySum += curSecurityObject.previousPercentChange;
                        end
                        
                        altIndustrySum = 0;
                        altIndustrySecurities = collect(keys(combinedSecuritySortedDateKeyDict[altIndustryKey][curDateKey]))
                        for m in 1:length(altIndustrySecurities)
                            altSecurityObjectKey = altIndustrySecurities[m]
                            altSecurityObject = combinedSecuritySortedDateKeyDict[altIndustryKey][curDateKey][altSecurityObjectKey]
                            
                            altIndustrySum += altSecurityObject.previousPercentChange;
                        end
                        
                        covarianceSum += ((curIndustrySum / length(curIndustrySecurities)) - curIndustryHistoricalReturn) * ((altIndustrySum / length(altIndustrySecurities)) - altIndustryHistoricalReturn)
                    else
                        skippedDays += 1
                    end
                    
                    covariance = covarianceSum / (length(curIndustryDateKeySet) - (1 + skippedDays))
                    IndustryCovarianceMatrix[curIndustryObjectId,altIndustryObjectId] = covariance;
                    IndustryCovarianceMatrix[altIndustryObjectId,curIndustryObjectId] = covariance;
                end
            end
        end
    end
end

## 6.C. Markets

In [ ]:
marketKeys = collect(keys(marketHistories))
numberMarkets = length(marketKeys)
MarketCovarianceMatrix = zeros(Float64, numberMarkets, numberMarkets)

combinedSecuritySortedDateKeyDict = Dict{String, Any}();

for i in 1:numberMarkets
    curMarketKey = marketKeys[i]
    curMarketObject = marketHistories[curMarketKey]
    curMarketObjectId = curMarketObject.marketId
    
    curMarketSecurityObjectKeys = curMarketObject.securityObjectKeys
    curMarketSecurityObjects = curMarketObject.securityObjects
    
    curMarketHistoricalReturn = marketHistoricalReturns[curMarketKey]
    
    if !haskey(combinedSecuritySortedDateKeyDict, curMarketKey)
        uniqueKeyDict = Dict{Float64, Any}();
        
        for j in 1:length(curMarketSecurityObjectKeys)
            curSecurityObjectKey = curMarketSecurityObjectKeys[j]
            curSecurityObject = curMarketSecurityObjects[curSecurityObjectKey]
            
            for k in 1:length(curSecurityObject.sortedDailyPerformanceObjectKeys)
                curDateKey = curSecurityObject.sortedDailyPerformanceObjectKeys[k]
                
                if haskey(uniqueKeyDict, curDateKey)
                    uniqueKeyDict[curDateKey][curSecurityObjectKey] = curSecurityObject.dailyPerformanceObjects[curDateKey]
                else
                    newDayDict = Dict{String, dailyPerformanceObject}();
                    newDayDict[curSecurityObjectKey] = curSecurityObject.dailyPerformanceObjects[curDateKey]
                    
                    uniqueKeyDict[curDateKey] = newDayDict;
                end
            end
        end
        
        combinedSecuritySortedDateKeyDict[curMarketKey] = uniqueKeyDict
    end
    
    for j in i:numberMarkets
        
        if (i == j) # Calculate the average variance across this industry's securities
            varianceSum = 0;
            curDateKeySet = sort(collect(keys(combinedSecuritySortedDateKeyDict[curMarketKey])))
            
            for k in 1:length(curDateKeySet)
                curDateKey = curDateKeySet[k]
                curDateSecurityObjectKeys = collect(keys(combinedSecuritySortedDateKeyDict[curMarketKey][curDateKey]))
                
                returnSum = 0;
                
                for m in 1:length(curDateSecurityObjectKeys)
                    curSecurityObjectKey = curDateSecurityObjectKeys[m]
                    curSecurityPerformanceObject = combinedSecuritySortedDateKeyDict[curMarketKey][curDateKey][curSecurityObjectKey]
                
                    curSecurityPreviousPercentChange = curSecurityPerformanceObject.previousPercentChange
                
                    returnSum += curSecurityPreviousPercentChange
                end
                
                varianceSum += ((returnSum / length(curDateSecurityObjectKeys)) - curMarketHistoricalReturn)^2
            end
            
            variance = varianceSum / (length(curDateKeySet) - 1)
            MarketCovarianceMatrix[curMarketObjectId, curMarketObjectId] = variance
        else # Calculate the covariance
            altMarketKey = marketKeys[j]
            altMarketObject = marketHistories[altMarketKey]
            altMarketObjectId = altMarketObject.marketId

            altMarketSecurityObjectKeys = altMarketObject.securityObjectKeys
            altMarketSecurityObjects = altMarketObject.securityObjects

            altMarketHistoricalReturn = marketHistoricalReturns[altMarketKey]
        
            
            if !haskey(combinedSecuritySortedDateKeyDict, altMarketKey)
                uniqueKeyDict = Dict{Float64, Any}();

                for k in 1:length(altMarketSecurityObjectKeys)
                    altSecurityObjectKey = altMarketSecurityObjectKeys[k]
                    altSecurityObject = altMarketSecurityObjects[altSecurityObjectKey]

                    for m in 1:length(altSecurityObject.sortedDailyPerformanceObjectKeys)
                        altDateKey = altSecurityObject.sortedDailyPerformanceObjectKeys[m]

                        if haskey(uniqueKeyDict, altDateKey)
                            uniqueKeyDict[altDateKey][altSecurityObjectKey] = altSecurityObject.dailyPerformanceObjects[altDateKey]
                        else
                            newDayDict = Dict{String, dailyPerformanceObject}();
                            newDayDict[altSecurityObjectKey] = altSecurityObject.dailyPerformanceObjects[altDateKey]

                            uniqueKeyDict[altDateKey] = newDayDict;
                        end
                    end
                end

                combinedSecuritySortedDateKeyDict[altMarketKey] = uniqueKeyDict
            end
            
            curMarketDateKeySet = sort(collect(keys(combinedSecuritySortedDateKeyDict[curMarketKey])))
            altMarketDateKeySet = sort(collect(keys(combinedSecuritySortedDateKeyDict[altMarketKey])))
        
            if (length(curMarketDateKeySet) > length(altMarketDateKeySet))
                covarianceSum = 0;
                skippedDays = 0;
                
                for k in 1:length(altMarketDateKeySet)
                    curDateKey = altMarketDateKeySet[k]
                    
                    if curDateKey in curMarketDateKeySet
                        
                        curMarketSum = 0;
                        curMarketSecurities = collect(keys(combinedSecuritySortedDateKeyDict[curMarketKey][curDateKey]))
                        for m in 1:length(curMarketSecurities)
                            curSecurityObjectKey = curMarketSecurities[m]
                            curSecurityObject = combinedSecuritySortedDateKeyDict[curMarketKey][curDateKey][curSecurityObjectKey]
                            
                            curMarketSum += curSecurityObject.previousPercentChange;
                        end
                        
                        altMarketSum = 0;
                        altMarketSecurities = collect(keys(combinedSecuritySortedDateKeyDict[altMarketKey][curDateKey]))
                        for m in 1:length(altMarketSecurities)
                            altSecurityObjectKey = altMarketSecurities[m]
                            altSecurityObject = combinedSecuritySortedDateKeyDict[altMarketKey][curDateKey][altSecurityObjectKey]
                            
                            altMarketSum += altSecurityObject.previousPercentChange;
                        end
                        
                        covarianceSum += ((curMarketSum / length(curMarketSecurities)) - curMarketHistoricalReturn) * ((altMarketSum / length(altMarketSecurities)) - altMarketHistoricalReturn)
                    else
                        skippedDays += 1
                    end
                    
                    covariance = covarianceSum / (length(altMarketDateKeySet) - (1 + skippedDays))
                    MarketCovarianceMatrix[curMarketObjectId,altMarketObjectId] = covariance;
                    MarketCovarianceMatrix[altMarketObjectId,curMarketObjectId] = covariance;
                end
            else
                covarianceSum = 0;
                skippedDays = 0;
                
                for k in 1:length(curMarketDateKeySet)
                    curDateKey = curMarketDateKeySet[k]
                    
                    if curDateKey in altMarketDateKeySet
                        
                        curMarketSum = 0;
                        curMarketSecurities = collect(keys(combinedSecuritySortedDateKeyDict[curMarketKey][curDateKey]))
                        for m in 1:length(curMarketSecurities)
                            curSecurityObjectKey = curMarketSecurities[m]
                            curSecurityObject = combinedSecuritySortedDateKeyDict[curMarketKey][curDateKey][curSecurityObjectKey]
                            
                            curMarketSum += curSecurityObject.previousPercentChange;
                        end
                        
                        altMarketSum = 0;
                        altMarketSecurities = collect(keys(combinedSecuritySortedDateKeyDict[altMarketKey][curDateKey]))
                        for m in 1:length(altMarketSecurities)
                            altSecurityObjectKey = altMarketSecurities[m]
                            altSecurityObject = combinedSecuritySortedDateKeyDict[altMarketKey][curDateKey][altSecurityObjectKey]
                            
                            altMarketSum += altSecurityObject.previousPercentChange;
                        end
                        
                        covarianceSum += ((curMarketSum / length(curMarketSecurities)) - curMarketHistoricalReturn) * ((altMarketSum / length(altMarketSecurities)) - altMarketHistoricalReturn)
                    else
                        skippedDays += 1
                    end
                    
                    covariance = covarianceSum / (length(curMarketDateKeySet) - (1 + skippedDays))
                    MarketCovarianceMatrix[curMarketObjectId,altMarketObjectId] = covariance;
                    MarketCovarianceMatrix[altMarketObjectId,curMarketObjectId] = covariance;Market
                end
            end
        end
    end
end

## 7. Compute Avg Return & Variance Vectors

### 7.A. Securities

In [ ]:
securityKeys = collect(keys(securityHistories))
SecurityReturnVector = zeros(numberSecurities)
SecurityVarianceVector = zeros(numberSecurities)

for i in 1:numberSecurities
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId
    
    curSecurityHistoricalReturn = securityHistoricalReturns[curSecurityKey]
    SecurityReturnVector[curSecurityObjectId] = curSecurityHistoricalReturn
    
    curSecurityVariance = SecurityCovarianceMatrix[curSecurityObjectId,curSecurityObjectId]
    SecurityVarianceVector[curSecurityObjectId] = curSecurityVariance
end

### 7.B. Industries

In [ ]:
industryKeys = collect(keys(industryHistories))
IndustryReturnVector = zeros(numberIndustries)
IndustryVarianceVector = zeros(numberIndustries)

for i in 1:numberIndustries
    curIndustryKey = industryKeys[i]
    curIndustryObject = industryHistories[curIndustryKey]
    curIndustryObjectId = curIndustryObject.industryId
    
    curIndustryHistoricalReturn = industryHistoricalReturns[curIndustryKey]
    IndustryReturnVector[curIndustryObjectId] = curIndustryHistoricalReturn
    
    curIndustryVariance = IndustryCovarianceMatrix[curIndustryObjectId,curIndustryObjectId]
    IndustryVarianceVector[curIndustryObjectId] = curIndustryVariance
end

### 7.C. Markets

In [ ]:
marketKeys = collect(keys(marketHistories))
MarketReturnVector = zeros(numberMarkets)
MarketVarianceVector = zeros(numberMarkets)

for i in 1:numberMarkets
    curMarketKey = marketKeys[i]
    curMarketObject = marketHistories[curMarketKey]
    curMarketObjectId = curMarketObject.marketId
    
    curMarketHistoricalReturn = marketHistoricalReturns[curMarketKey]
    MarketReturnVector[curMarketObjectId] = curMarketHistoricalReturn
    
    curMarketVariance = MarketCovarianceMatrix[curMarketObjectId,curMarketObjectId]
    MarketVarianceVector[curMarketObjectId] = curMarketVariance
end

## 8. Define Optimization Model

In [ ]:
using JuMP, Ipopt, LinearAlgebra

function findOptimalPortfolio(AverageReturnVector, CovarianceMatrix, λ, allocationLimit)
    m = Model(Ipopt.Optimizer)
    
    retVector = convert(Vector{Float64}, AverageReturnVector[1:length(AverageReturnVector), 1])
    covMatrix = convert(Matrix{Float64}, CovarianceMatrix[1:length(AverageReturnVector), 1:length(AverageReturnVector)])
    
    @variable(m, x[1:length(AverageReturnVector)] >= 0)
    
    @constraint(m, sum(x) == 1)
    for i in 1:length(AverageReturnVector)
        @constraint(m, x[i] <= allocationLimit)
    end
    
    @objective(m, Min, (λ*sum(x[i] * covMatrix[i,j] * x[j] for i in 1:length(AverageReturnVector), j in 1:length(AverageReturnVector))) - sum(retVector[i] * x[i] for i in 1:length(AverageReturnVector)))
    optimize!(m)
    
    xsol = value.(x)
    return (xsol)
end
;

## 9. Compute Sample Results

### 9.A. Securities

In [ ]:
λ = 1
(x_optimal_securities_1) = findOptimalPortfolio(SecurityReturnVector, SecurityCovarianceMatrix, λ, SECURITY_ALLOCATION_LIMIT)
interestingX_securities_1 = findall(x_optimal_securities_1 .> 1e-4)
ret_securities_1 = dot(SecurityReturnVector,x_optimal_securities_1)
std_securities_1 = sqrt(dot(x_optimal_securities_1,SecurityCovarianceMatrix*x_optimal_securities_1))

λ = 3e-2
(x_optimal_securities_3e_2) = findOptimalPortfolio(SecurityReturnVector, SecurityCovarianceMatrix, λ, SECURITY_ALLOCATION_LIMIT)
interestingX_securities_3e_2 = findall(x_optimal_securities_3e_2 .> 1e-4)
ret_securities_3e_2 = dot(SecurityReturnVector,x_optimal_securities_3e_2)
std_securities_3e_2 = sqrt(dot(x_optimal_securities_3e_2,SecurityCovarianceMatrix*x_optimal_securities_3e_2))

### 9.B. Industries

In [ ]:
λ = 1
(x_optimal_industries_1) = findOptimalPortfolio(IndustryReturnVector, IndustryCovarianceMatrix, λ, INDUSTRY_ALLOCATION_LIMIT)
interestingX_industries_1 = findall(x_optimal_industries_1 .> 1e-4)
ret_industries_1 = dot(IndustryReturnVector,x_optimal_industries_1)
std_industries_1 = sqrt(dot(x_optimal_industries_1,IndustryCovarianceMatrix*x_optimal_industries_1))

λ = 3e-2
(x_optimal_industries_3e_2) = findOptimalPortfolio(IndustryReturnVector, IndustryCovarianceMatrix, λ, INDUSTRY_ALLOCATION_LIMIT)
interestingX_industries_3e_2 = findall(x_optimal_industries_3e_2 .> 1e-4)
ret_industries_3e_2 = dot(IndustryReturnVector,x_optimal_industries_3e_2)
std_industries_3e_2 = sqrt(dot(x_optimal_industries_3e_2,IndustryCovarianceMatrix*x_optimal_industries_3e_2))

### 9.C. Markets

In [ ]:
λ = 1
(x_optimal_markets_1) = findOptimalPortfolio(MarketReturnVector, MarketCovarianceMatrix, λ, MARKET_ALLOCATION_LIMIT)
interestingX_markets_1 = findall(x_optimal_markets_1 .> 1e-4)
ret_markets_1 = dot(MarketReturnVector,x_optimal_markets_1)
std_markets_1 = sqrt(dot(x_optimal_markets_1,MarketCovarianceMatrix*x_optimal_markets_1))

λ = 3e-2
(x_optimal_markets_3e_2) = findOptimalPortfolio(MarketReturnVector, MarketCovarianceMatrix, λ, MARKET_ALLOCATION_LIMIT)
interestingX_markets_3e_2 = findall(x_optimal_markets_3e_2 .> 1e-4)
ret_markets_3e_2 = dot(MarketReturnVector,x_optimal_markets_3e_2)
std_markets_3e_2 = sqrt(dot(x_optimal_markets_3e_2,MarketCovarianceMatrix*x_optimal_markets_3e_2))

## 10. Compute Paredo Curves

### 10.A. Securities

In [ ]:
N = 50
ret_securities_paredo = zeros(N)
std_securities_paredo = zeros(N)
lambda_values = exp10.(range(-3,stop=4,length=N))

for (i,λ) in enumerate(lambda_values)
    
    xsol = findOptimalPortfolio(SecurityReturnVector, SecurityCovarianceMatrix, λ, SECURITY_ALLOCATION_LIMIT)

    println(i)
    
    ret_securities_paredo[i] = dot(SecurityReturnVector,xsol)
    std_securities_paredo[i] = sqrt(dot(xsol,SecurityCovarianceMatrix*xsol))
end

### 10.B. Industries

In [ ]:
N = 50
ret_industries_paredo = zeros(N)
std_industries_paredo = zeros(N)
lambda_values = exp10.(range(-3,stop=4,length=N))
lambdas = zeros(N);

highestIndustryReturn = 0;
highestIndustryReturnIndex = -1;
bestIndustryLambda = 0;

for (i,λ) in enumerate(lambda_values)
    
    xsol = findOptimalPortfolio(IndustryReturnVector, IndustryCovarianceMatrix, λ, INDUSTRY_ALLOCATION_LIMIT)

    println(i)
    
    ret_industries_paredo[i] = dot(IndustryReturnVector,xsol)
    std_industries_paredo[i] = sqrt(dot(xsol,IndustryCovarianceMatrix*xsol))
    
    if ret_industries_paredo[i] > highestIndustryReturn
        highestIndustryReturn = ret_industries_paredo[i]
        highestIndustryReturnIndex = i;
        bestIndustryLambda = λ;
    end
end

### 10.C. Markets

In [ ]:
N = 50
ret_markets_paredo = zeros(N)
std_markets_paredo = zeros(N)
lambda_values = exp10.(range(-3,stop=4,length=N))
lambdas = zeros(N);

highestMarketReturn = 0;
highestMarketReturnIndex = -1;
bestMarketLambda = 0;

for (i,λ) in enumerate(lambda_values)
    
    xsol = findOptimalPortfolio(MarketReturnVector, MarketCovarianceMatrix, λ, MARKET_ALLOCATION_LIMIT)

    println(i)
    
    ret_markets_paredo[i] = dot(MarketReturnVector,xsol)
    std_markets_paredo[i] = sqrt(dot(xsol,MarketCovarianceMatrix*xsol))
    
    if ret_markets_paredo[i] > highestMarketReturn
        highestMarketReturn = ret_markets_paredo[i]
        highestMarketReturnIndex = i;
        bestMarketLambda = λ;
    end
end

## 11. Combined Optimization

Note: This is where it gets to be an exploration, there is no one way to do this.
Some ideas:
- Optimize over markets -> Optimize over industries -> Optimize over securities
- Optimize over industries -> Optimize over securities

### 11.A. Industry -> Security

In [ ]:
industry_allocation_solution_vector = findOptimalPortfolio(IndustryReturnVector, IndustryCovarianceMatrix, bestIndustryLambda, INDUSTRY_ALLOCATION_LIMIT)

numLambdaBuckets = 50
CombinedOptimizationReturn = zeros(numLambdaBuckets)
CombinedOptimizationStd = zeros(numLambdaBuckets)
lambdaValues = exp10.(range(-3,stop=4,length=numLambdaBuckets))

highestReturn = 0;
highestReturnIndex = 0;
highestReturnLambda = 0;
CombinedOptimizationSecurityAllocationVector = zeros(numLambdaBuckets, numberSecurities)

for (n,λ) in enumerate(lambdaValues)
    totalSecuritySum = 0;
    for i in 1:length(industry_allocation_solution_vector)
        curAllocationPercentage = industry_allocation_solution_vector[i]
        if (curAllocationPercentage > 1e-4)
            # This industry has a significant allocation, optimize its securities
            curIndustryKey = industryIdMap[i];
            curIndustryObject = industryHistories[curIndustryKey]
            securityIdVector = Vector{Int64}();
            
            curIndustrySecurityObjectKeys = curIndustryObject.securityObjectKeys
            curIndustrySecurityObjects = curIndustryObject.securityObjects
            
            for j in 1:length(curIndustrySecurityObjectKeys)
                curSecurityObjectKey = curIndustrySecurityObjectKeys[j];
                curSecurity = curIndustrySecurityObjects[curSecurityObjectKey];
                push!(securityIdVector, curSecurity.tickerId)
            end

            returnVector = zeros(length(securityIdVector))
            covarianceMatrix = zeros(Float64, length(securityIdVector), length(securityIdVector))

            for j in 1:length(securityIdVector)
                curId = securityIdVector[j];
                curSecurityKey = securityIdMap[curId]
                curSecurityReturn = SecurityReturnVector[curId]
                returnVector[j] = curSecurityReturn

                for k in j:length(securityIdVector)
                    altId = securityIdVector[k];
                    curSecurityCovariance = SecurityCovarianceMatrix[curId,altId]

                    covarianceMatrix[j, k] = curSecurityCovariance
                    covarianceMatrix[k, j] = curSecurityCovariance
                end
            end

            industry_xsol = findOptimalPortfolio(returnVector, covarianceMatrix, λ, SECURITY_ALLOCATION_LIMIT)
            totalSecuritySum += length(securityIdVector)
            println(n)
            println(industry_xsol)
            
            # There is a bug somewhere below here resulting in only 33% final allocation
            # Think it has something to do with overwriting the CombinedAllocVector
            
            for j in 1:length(securityIdVector)
                curId = securityIdVector[j];
                curSecurityAllocation = industry_xsol[j]
                if (curSecurityAllocation > 1e-8)
                    CombinedOptimizationSecurityAllocationVector[n, curId] = curSecurityAllocation * curAllocationPercentage;
                end
            end
        else
           # This industry does not have any substantial allocation, skip
        end
    end
    println(totalSecuritySum)
    
    CombinedOptimizationReturn[n] = dot(SecurityReturnVector,CombinedOptimizationSecurityAllocationVector[n,:])
    CombinedOptimizationStd[n] = sqrt(dot(CombinedOptimizationSecurityAllocationVector[n,:],SecurityCovarianceMatrix*CombinedOptimizationSecurityAllocationVector[n,:]))
    if (CombinedOptimizationReturn[n] > highestReturn)
        highestReturn = CombinedOptimizationReturn[n];
        highestReturnIndex = n;
        highestReturnLambda = λ;
    end
end

optimalCombinedAllocationVector = CombinedOptimizationSecurityAllocationVector[highestReturnIndex,:]
interestingCombinedSecurities = findall(optimalCombinedAllocationVector .> 1e-4);

## 12. Print Combined Optimization Results

### 12.A. Industry -> Security

In [ ]:
# println("Optimal Asset Selection for Industry -> Security Optimization:")
println("================================")
for i in 1:length(interestingCombinedSecurities)
    curSecurityId = interestingCombinedSecurities[i]
    curSecurityAllocation = optimalCombinedAllocationVector[curSecurityId]
    
    allKeys = collect(keys(securityHistories))
    for j in 1:length(allKeys)
        curKey = allKeys[j]
        curSecurityObject = securityHistories[curKey]
        curSecurityObjectId = curSecurityObject.tickerId
        
        if (curSecurityId == curSecurityObjectId)
            curSecurityObjectTicker = curSecurityObject.tickerSymbol
            curSecurityObjectName = curSecurityObject.securityName 
            curSecurityObjectIndustry = curSecurityObject.industry
            curSecurityObjectMarket = curSecurityObject.market
            curSecurityObjectAverageReturn = securityHistoricalReturns[curSecurityObjectTicker]
            curSecurityObjectAverageStdDev = sqrt(SecurityCovarianceMatrix[curSecurityId,curSecurityId])
            
            println()
            print("Security ID: ")
            println(curSecurityId)
            print("Security Name: ")
            println(curSecurityObjectName)
            print("Ticker Symbol: ")
            println(curSecurityObjectTicker)
            print("Portfolio Allocation (%): ")
            println(100*curSecurityAllocation)
            print("Market: ")
            println(curSecurityObjectMarket)
            print("Industry: ")
            println(curSecurityObjectIndustry)
            print("Historical Average Daily Return (%): ")
            println(100*curSecurityObjectAverageReturn)
            print("Historical Average Daily Return Standard Deviation (%): ")
            println(100*curSecurityObjectAverageStdDev)
        end 
    end
end

println()
print("Expected Daily Return: ")
println(CombinedOptimizationReturn[highestReturnIndex]*100, "(%)")
print("Expected Annualized Return: ")
println((((1 + CombinedOptimizationReturn[highestReturnIndex])^365)-1)*100, "(%)")
;

# Monte Carlo Simulation & CVaR Optimization

## 1. Define Structs and Functions

In [ ]:
using Distributions, JuMP, Ipopt, LinearAlgebra

mutable struct monteCarloPerformanceObject
    tickerId
    tickerSymbol
    sortedDailyPerformanceObjectKeys
    dailyPeriodicReturnMap
    average
    variance
    stdDev
    drift
end

mutable struct monteCarloSimulationHistogramObject
    tickerId
    tickerSymbol
    simulationHistogram
    numberDataPoints
    averageLoss
end

function monteCarloSimulation(drift, avg, variance, stdDev, startPrice, duration)
    standardNormal = Normal() # default is mean = 0, std = 1
    curPrice = startPrice
    
    for i in 1:duration
        randomSeed = rand();
        dailyVolatility = stdDev * quantile(standardNormal, randomSeed)
        nextPrice = curPrice * exp(drift + dailyVolatility)
        curPrice = nextPrice
    end
    
    return curPrice
end

function valueAtRisk(x::Vector{Float64}, f::Vector{Float64}, a::Float64)
    i = findfirst(p -> p >= a, cumsum(f))
    if i === nothing
        return x[end]
    else
        return x[i]
    end
end
    
function conditionalValueAtRisk(x::Vector{Float64}, f::Vector{Float64}, a::Float64)
    valAtRisk = valueAtRisk(x, f, a)
    if iszero(a)
        return valAtRisk
    else
        tail = x .>= valAtRisk
        return (sum(x[tail] .* f[tail]) / (1 - a))
    end
end

function optimizeOverConditionalValueAtRisk(yExpectationVect, yMatrix, a, numSimulations, returnMinimum)
    m = Model(Ipopt.Optimizer)
    
    # Desired portfolio positions
    @variable(m, .05 >= x[1:length(yExpectationVect)] >= 0)
    # auxillary variables for reduction to convex programming
    @variable(m, u[1:numSimulations] >= 0)
    @variable(m, lambda)

    # Sum of positions adds to one
    @constraint(m, sum(x[i] for i in 1:length(yExpectationVect)) == 1)
    # Only allow portfolios with expected return at least returnMinimum
    @constraint(m, -1 * sum(x[i] * yExpectationVect[i] for i in 1:length(yExpectationVect)) <= -1 * returnMinimum)
    # Reduction to convex programming
    for j in 1:numSimulations
         @constraint(m, sum(x[i] * yMatrix[i,j] + lambda + u[j] for i in 1:length(yExpectationVect)) >= 0)  
    end
    
    @objective(m, Min, lambda + (1/(numSimulations*(1-a))) * sum(u[k] for k in 1:numSimulations))
    optimize!(m)
    
    xsol = value.(x)
    return (xsol, value.(lambda))
end
;

## 2. Create the Monte Carlo Performance Objects, find Average, Variance, Std Dev. of Periodic Daily Time Series Returns

In [ ]:
# Create the monteCarloPerformanceObject objects
securityKeys = collect(keys(securityHistories))
SecurityPeriodicTimeSeriesDict = Dict{String, monteCarloPerformanceObject}();
for i in 1:length(securityKeys)
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId;
    curSecurityObjectTickerSymbol = curSecurityObject.tickerSymbol
    curSecurityObjectSortedDailyPerformanceObjectKeys = sort(collect(curSecurityObject.sortedDailyPerformanceObjectKeys))
    curSecurityObjectDailyPerformanceObjects = curSecurityObject.dailyPerformanceObjects
    
    newPeriodicDict = Dict{Float64, Float64}();
    
    newObject = monteCarloPerformanceObject(
        curSecurityObjectId,
        curSecurityObjectTickerSymbol,
        curSecurityObjectSortedDailyPerformanceObjectKeys,
        newPeriodicDict,
        0,
        0,
        0,
        0
    );
    
    firstDateKey = curSecurityObjectSortedDailyPerformanceObjectKeys[1];
    newObject.dailyPeriodicReturnMap[firstDateKey] = 0;
    
    for j in 2:length(curSecurityObjectSortedDailyPerformanceObjectKeys)
        curDateKey = curSecurityObjectSortedDailyPerformanceObjectKeys[j];
        curDailyPerformanceObject = curSecurityObjectDailyPerformanceObjects[curDateKey];
        curDailyPerformanceObectClose = curDailyPerformanceObject.closeUSD;
        prevDateKey = curSecurityObjectSortedDailyPerformanceObjectKeys[j-1];
        prevDailyPerformanceObject = curSecurityObjectDailyPerformanceObjects[prevDateKey];
        prevDailyPerformanceObectClose = prevDailyPerformanceObject.closeUSD;
        
        periodicDailyReturn = log(curDailyPerformanceObectClose / prevDailyPerformanceObectClose)
        newObject.dailyPeriodicReturnMap[curDateKey] = periodicDailyReturn
    end
    
    SecurityPeriodicTimeSeriesDict[curSecurityKey] = newObject;
end

# Find the average
for i in 1:length(securityKeys)
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId;
    curSecurityObjectTickerSymbol = curSecurityObject.tickerSymbol
    
    curSecurityDailyPeriodicTimeSeries = SecurityPeriodicTimeSeriesDict[curSecurityKey]
    curSecurityDailyPeriodicTimeSeriesDateKeys = curSecurityDailyPeriodicTimeSeries.sortedDailyPerformanceObjectKeys
    curSecurityDailyPeriodicTimeSeriesReturnMap = curSecurityDailyPeriodicTimeSeries.dailyPeriodicReturnMap
    
    curAverageSum = 0;
    
    for j in 1:length(curSecurityDailyPeriodicTimeSeriesDateKeys)
        curDateKey = curSecurityDailyPeriodicTimeSeriesDateKeys[j]
        curPeriodicReturn = curSecurityDailyPeriodicTimeSeriesReturnMap[curDateKey];
        curAverageSum += curPeriodicReturn
    end

    SecurityPeriodicTimeSeriesDict[curSecurityKey].average = (curAverageSum / length(curSecurityDailyPeriodicTimeSeriesDateKeys))
end

# Find the Variance / Std Dev
for i in 1:length(securityKeys)
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId;
    curSecurityObjectTickerSymbol = curSecurityObject.tickerSymbol
    
    curSecurityDailyPeriodicTimeSeries = SecurityPeriodicTimeSeriesDict[curSecurityKey]
    curSecurityDailyPeriodicTimeSeriesDateKeys = curSecurityDailyPeriodicTimeSeries.sortedDailyPerformanceObjectKeys
    curSecurityDailyPeriodicTimeSeriesReturnMap = curSecurityDailyPeriodicTimeSeries.dailyPeriodicReturnMap
    curSecurityDailyPeriodicTimeSeriesAverage = curSecurityDailyPeriodicTimeSeries.average
    
    curVarianceSum = 0;
    
    for j in 1:length(curSecurityDailyPeriodicTimeSeriesDateKeys)
        curDateKey = curSecurityDailyPeriodicTimeSeriesDateKeys[j]
        curPeriodicReturn = curSecurityDailyPeriodicTimeSeriesReturnMap[curDateKey];
        curVarianceSum += (curPeriodicReturn - curSecurityDailyPeriodicTimeSeriesAverage)^2
    end

    varianceValue = (curVarianceSum / (length(curSecurityDailyPeriodicTimeSeriesDateKeys) - 1))
    SecurityPeriodicTimeSeriesDict[curSecurityKey].variance = varianceValue;
    SecurityPeriodicTimeSeriesDict[curSecurityKey].stdDev = sqrt(varianceValue);
    SecurityPeriodicTimeSeriesDict[curSecurityKey].drift = curSecurityDailyPeriodicTimeSeriesAverage - (varianceValue/2);
end

## 3. Perform The Monte Carlo Simulation

In [ ]:
# Perform the simulation
numberSimulations = 1000
MonteCarloSimulationResults = zeros(length(securityKeys), numberSimulations)
MaximumSimulatedPercentChange = -1000000;
MinimumSimulatedPercentChange = 1000000;
for i in 1:length(securityKeys)
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId;
    curSecurityObjectTickerSymbol = curSecurityObject.tickerSymbol
    curSecurityObjectSortedDailyPerformanceObjectKeys = sort(collect(curSecurityObject.sortedDailyPerformanceObjectKeys))
    curSecurityObjectDailyPerformanceObjects = curSecurityObject.dailyPerformanceObjects
    
    curSecurityDailyPeriodicTimeSeries = SecurityPeriodicTimeSeriesDict[curSecurityKey]
    curSecurityDailyPeriodicTimeSeriesAverage = curSecurityDailyPeriodicTimeSeries.average
    curSecurityDailyPeriodicTimeSeriesVariance = curSecurityDailyPeriodicTimeSeries.variance
    curSecurityDailyPeriodicTimeSeriesStdDev = curSecurityDailyPeriodicTimeSeries.stdDev
    curSecurityDailyPeriodicTimeSeriesDrift = curSecurityDailyPeriodicTimeSeries.drift
    
    finalDateKey = curSecurityObjectSortedDailyPerformanceObjectKeys[length(curSecurityObjectSortedDailyPerformanceObjectKeys)]
    curSecurityClosePrice = curSecurityObjectDailyPerformanceObjects[finalDateKey].closeUSD
    
    
    for j in 1:numberSimulations
        curMonteCarloSimulationResult = monteCarloSimulation(
            curSecurityDailyPeriodicTimeSeriesDrift,
            curSecurityDailyPeriodicTimeSeriesAverage,
            curSecurityDailyPeriodicTimeSeriesVariance,
            curSecurityDailyPeriodicTimeSeriesStdDev,
            curSecurityClosePrice,
            90
        )
        
        curPercentChangeOverall = round(-1* ((curMonteCarloSimulationResult - curSecurityClosePrice) / curSecurityClosePrice), digits=8)
        MonteCarloSimulationResults[curSecurityObjectId, j] = curPercentChangeOverall;
            
        if (curPercentChangeOverall > MaximumSimulatedPercentChange)
            MaximumSimulatedPercentChange = curPercentChangeOverall
        end
        
        if (curPercentChangeOverall < MinimumSimulatedPercentChange)
            MinimumSimulatedPercentChange = curPercentChangeOverall
        end
    end
end

## 4. Consolidate Results

### 4.1 Create the General Histogram

In [ ]:
# Create the histogram
NumberBuckets = 1000
TotalSimulationReturnSpread = MaximumSimulatedPercentChange - MinimumSimulatedPercentChange;
SimulationHistogramBucketWidth = TotalSimulationReturnSpread / NumberBuckets;
HistogramBucketBoundaries = zeros(NumberBuckets, 3)
curBucketMinimum = MinimumSimulatedPercentChange
for i in 1:NumberBuckets
    HistogramBucketBoundaries[i,1] = curBucketMinimum;
    curBucketMax = curBucketMinimum + SimulationHistogramBucketWidth
    HistogramBucketBoundaries[i,2] = curBucketMax;
    curBucketAverage = ((curBucketMinimum + curBucketMax) / 2)
    HistogramBucketBoundaries[i,3] = curBucketAverage;
    curBucketMinimum = curBucketMax;
end

### 4.2 Populate the Histogram

In [ ]:
# Consolidate results
# What do we want? - A histogram of performance

MonteCarloHistogramCollection = Dict{String, monteCarloSimulationHistogramObject}();
for i in 1:length(securityKeys)
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId;
    
    curSecurityDailyPeriodicTimeSeries = SecurityPeriodicTimeSeriesDict[curSecurityKey]
    curSecurityDailyPeriodicTimeSeriesAverage = curSecurityDailyPeriodicTimeSeries.average
    curSecurityDailyPeriodicTimeSeriesVariance = curSecurityDailyPeriodicTimeSeries.variance
    curSecurityDailyPeriodicTimeSeriesStdDev = curSecurityDailyPeriodicTimeSeries.stdDev
    curSecurityDailyPeriodicTimeSeriesDrift = curSecurityDailyPeriodicTimeSeries.drift
    
    curSecurityMonteCarloPerformanceResultsVector = MonteCarloSimulationResults[curSecurityObjectId, :]
    
    curSecuritySimulationHistogramCounts = zeros(NumberBuckets);
    average = 0;
    for j in 1:NumberBuckets
        curBucketCount = length(findall(HistogramBucketBoundaries[j,1] .<= curSecurityMonteCarloPerformanceResultsVector .< HistogramBucketBoundaries[j,2]))
        curSecuritySimulationHistogramCounts[j] = curBucketCount
        average += curBucketCount * HistogramBucketBoundaries[j,3];
    end
    average = average / NumberBuckets
    
    newSimulationHistogramObject = monteCarloSimulationHistogramObject(
        curSecurityObjectId,
        curSecurityKey,
        curSecuritySimulationHistogramCounts,
        sum(curSecuritySimulationHistogramCounts),
        average
    )
    
    MonteCarloHistogramCollection[curSecurityKey] = newSimulationHistogramObject
end

### 4.3 Compute Average Losses, Drift, Std Dev

In [ ]:
MonteCarloSimulationAverageLosses = zeros(length(securityKeys))
for i in 1:length(securityKeys)
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId;
    
    curAvgLoss = MonteCarloHistogramCollection[curSecurityKey].averageLoss
    MonteCarloSimulationAverageLosses[curSecurityObjectId] = curAvgLoss
end

PeriodicReturnAverage = zeros(length(securityKeys))
PeriodicReturnStdDev = zeros(length(securityKeys))
PeriodicReturnDrift = zeros(length(securityKeys))
for i in 1:length(securityKeys)
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId;
    curSecurityObjectTickerSymbol = curSecurityObject.tickerSymbol
    
    curPeriodicTimeSeriesObject = SecurityPeriodicTimeSeriesDict[curSecurityKey]
    PeriodicReturnAverage[curSecurityObjectId] = curPeriodicTimeSeriesObject.average
    PeriodicReturnStdDev[curSecurityObjectId] = curPeriodicTimeSeriesObject.stdDev
    PeriodicReturnDrift[curSecurityObjectId] = curPeriodicTimeSeriesObject.drift
end

### 4.4 Compute VaR and CVaR for Individual Securities

In [ ]:
# Get the ValueAtRisk and ConditionalValueAtRisk of all Samples based on the monte carlo simulation

SecurityValueAtRisk = zeros(length(securityKeys), 3)
SecurityConditionalValueAtRisk = zeros(length(securityKeys), 3)
for i in 1:length(securityKeys)
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId;
    
    curSecurityHistogramCollectionObject = MonteCarloHistogramCollection[curSecurityKey]
    curSecurityHistogram = curSecurityHistogramCollectionObject.simulationHistogram
    curSecurityHistogramNumberPoints = curSecurityHistogramCollectionObject.numberDataPoints
      
    a = [.99, .95, .90];
    counter = 1;
    for b in a
        curSecurityVAR = valueAtRisk(HistogramBucketBoundaries[:,3], curSecurityHistogram./curSecurityHistogramNumberPoints, b)
        SecurityValueAtRisk[curSecurityObjectId, counter] = curSecurityVAR
        curSecurityCOVAR = conditionalValueAtRisk(HistogramBucketBoundaries[:,3], curSecurityHistogram./curSecurityHistogramNumberPoints, b)
        SecurityConditionalValueAtRisk[curSecurityObjectId, counter] = curSecurityCOVAR
        counter += 1;
    end
end

### 4.5 Consolidate Data Needed for Optimization

In [ ]:
# Consolidate the building blocks for the optimization

confidenceLevel = 0.95
riskTolerance = .5 
yExpectationVector = zeros(length(securityKeys))
yMatrix = -1 * MonteCarloSimulationResults

for i in 1:length(securityKeys)
    curSecurityKey = securityKeys[i]
    curSecurityObject = securityHistories[curSecurityKey]
    curSecurityObjectId = curSecurityObject.tickerId;
    
    yExpectationVector[curSecurityObjectId] = -1 * MonteCarloSimulationAverageLosses[curSecurityObjectId]
    
    curSecurityHistogramObject = MonteCarloHistogramCollection[curSecurityKey]
    curSecurityHistogram = curSecurityHistogramObject.simulationHistogram
    curSecurityHistogramNumberPoints = curSecurityHistogramObject.numberDataPoints
end

### 4.6 Perform the 2-Step Optimization

In [ ]:
# Calculate optimal portfolio for first time
(x_sol, lambda) = optimizeOverConditionalValueAtRisk(yExpectationVector, yMatrix, confidenceLevel, numberSimulations, 0)

# Compile only the interesting securities >= .01 to re-optimize over
count = 0
for i in 1:length(x_sol)
    if (x_sol[i] >= .01)
       count += 1 
    end
end

interestingSecurities = zeros(count)
count = 1;
for i in 1:length(x_sol)
    if (x_sol[i] >= .01)
        interestingSecurities[count] = convert(Int64,i)
        count += 1 
    end
end

newYExpectationVector = zeros(count)
newYMatrix = zeros(count, numberSimulations)
for i in 1:length(interestingSecurities)
    curIndex = convert(Int64, interestingSecurities[i])
    newYExpectationVector[i] = yExpectationVector[curIndex]
    newYMatrix[i,:] = yMatrix[curIndex, :]
end

# Calculate the optimal portfolio over the interesting securities
(x_sol_2, lambda_2) = optimizeOverConditionalValueAtRisk(newYExpectationVector, newYMatrix, confidenceLevel, numberSimulations, 0);


### 4.7 Print Results of Monte Carlo Simulation Optimization

In [ ]:
averageReturnSum = 0;
OptimalPortfolioSimulationHistories = zeros(length(x_sol_2), numberSimulations)
for i in 1:length(x_sol_2)
   if (x_sol_2[i] >= .01)
        curSecurityId = convert(Int64,interestingSecurities[i])
        curSecurityKey = securityIdMap[curSecurityId]
        curSecurityObject = securityHistories[curSecurityKey]
        
        curSecurityObjectTicker = curSecurityObject.tickerSymbol
        curSecurityObjectName = curSecurityObject.securityName 
        curSecurityObjectIndustry = curSecurityObject.industry
        curSecurityObjectMarket = curSecurityObject.market
        curSecurityObjectAverageReturn = securityHistoricalReturns[curSecurityObjectTicker]
        curSecurityObjectAverageStdDev = sqrt(SecurityCovarianceMatrix[curSecurityId,curSecurityId])
        
        curSecurityAllocation = x_sol_2[i]
        
        averageReturnSum += -1*MonteCarloSimulationAverageLosses[curSecurityId] * curSecurityAllocation
        
        curSecuritySimulationResults = MonteCarloSimulationResults[curSecurityId,:]
        OptimalPortfolioSimulationHistories[i,:] = curSecuritySimulationResults
        
        println()
        print("Security ID: ")
        println(curSecurityId)
        print("Security Name: ")
        println(curSecurityObjectName)
        print("Ticker Symbol: ")
        println(curSecurityObjectTicker)
        print("Portfolio Allocation (%): ")
        println(100*curSecurityAllocation)
        print("Market: ")
        println(curSecurityObjectMarket)
        print("Industry: ")
        println(curSecurityObjectIndustry)
        print("Historical Average Daily Return (%): ")
        println(100*curSecurityObjectAverageReturn)
        print("Historical Average Daily Return Standard Deviation (%): ")
        println(100*curSecurityObjectAverageStdDev)
        print("Monte Carlo Simulation Average Loss (%): ")
        println(100*MonteCarloSimulationAverageLosses[curSecurityId])
    end
end

# F = a + 1/q(1-b) sum(x^T * y_i - a)+
count = 0;
conditionalLossSum = 0;
for i in 1:numberSimulations
    curSimulationSecurityExpectations = OptimalPortfolioSimulationHistories[:,i]
    curSum = 0;
    for j in 1:length(curSimulationSecurityExpectations)
        curSum += curSimulationSecurityExpectations[j] * x_sol_2[j]
    end

    if (curSum - lambda_2 >= 0)
        conditionalLossSum += curSum - lambda_2
    end
end


println()
print("Portfolio 95%-Value At Risk (Loss %): ")
println(100*lambda_2)
print("Portfolio 95%-Conditional Value At Risk (Loss %): ")
println(100*(lambda_2 + conditionalLossSum/(numberSimulations*.05)))
print("Portfolio 3-Month Expected Return (%): ")
println(100*averageReturnSum)


### 4.8 Compute the VaR and CVaR of MV Portfolio For Comparison

In [ ]:
mvPortfolioSimulationIterationLoss = zeros(numberSimulations)
count = 0;
maxLoss = -100000
minLoss = 1000000
for j in 1:numberSimulations
    curExpectedLoss = MonteCarloSimulationResults[:,j]
    
    expectedLossSum = 0;
    for i in 1:length(interestingCombinedSecurities)
        curSecurityId = interestingCombinedSecurities[i]
        curSecurityAllocation = optimalCombinedAllocationVector[curSecurityId]
        curSecurityExpectedLoss = curSecurityAllocation * curExpectedLoss[curSecurityId]
        expectedLossSum += curSecurityExpectedLoss
    end
    
    if (expectedLossSum > maxLoss)
        maxLoss = expectedLossSum
    end
    
    if (expectedLossSum < minLoss)
        minLoss = expectedLossSum
    end
    
    mvPortfolioSimulationIterationLoss[j] = expectedLossSum
end

mvLossSpread = maxLoss - minLoss
mvHistBucketWidth = mvLossSpread / 1000
mvHist = zeros(1000)

sortedMvPortfolioSimulationLosses = sort(mvPortfolioSimulationIterationLoss)

for i in 1:numberSimulations
    curSimulationLoss = sortedMvPortfolioSimulationLosses[i]
    
    curBucket = -1;
    for j in 1:1000
        curBucketMaximum = minLoss + (j * mvHistBucketWidth);
        if (curSimulationLoss < curBucketMaximum)
            curBucket = j
            break
        end
        
    end
    
    mvHist[curBucket] += 1/numberSimulations
end

sortedMvPortfolioSimulalationLossProbabilities = zeros(length(sortedMvPortfolioSimulationLosses))
for i in 1:numberSimulations
    sortedMvPortfolioSimulalationLossProbabilities[i] = .001;
end


MVVaR = valueAtRisk(sortedMvPortfolioSimulationLosses, sortedMvPortfolioSimulalationLossProbabilities, 0.95)
MVCVaR = conditionalValueAtRisk(sortedMvPortfolioSimulationLosses, sortedMvPortfolioSimulalationLossProbabilities, 0.95)

print("MV Portfolio VaR (% Loss): ")
println(100*MVVaR)
print("MV Portfolio CVaR (% Loss): ")
println(100*MVCVaR)

# Figures & Plots

## 1. Sorted Daily Return & Standard Deviation

### 1.A. Securities

In [ ]:
using PyPlot

# Obtain the sorted permutation of each security based on return
sortedSecurities = sortperm(SecurityReturnVector)

# Plot the Security Standard Dev and Return in sorted order
fig = figure(figsize=(12,6))
subplot(211)
xlim(0, numberSecurities)
plot(100*sqrt.(SecurityVarianceVector)[sortedSecurities], "g")
ylabel("Daily Standard Deviation (%)")
title("Daily Standard Deviation and Expected Daily Return of all Securities")
tight_layout()

subplot(212)
xlim(0, numberSecurities)
plot(100*SecurityReturnVector[sortedSecurities])
plot([0,numberSecurities],[0,0],"r--")
ylabel("Expected Daily Return (%)")
tight_layout()
savefig("images/SecuritySortedReturn.jpg")
;

### 1.B. Industries

In [ ]:
using PyPlot

# Obtain the sorted permutation of each industry based on return
sortedIndustries = sortperm(IndustryReturnVector)

# Plot the Industry Standard Dev and Return in sorted order
fig = figure(figsize=(12,6))
subplot(211)
xlim(0, numberIndustries-1)
plot(100*sqrt.(IndustryVarianceVector)[sortedIndustries], "g")
ylabel("Daily Standard Deviation (%)")
title("Daily Standard Deviation and Expected Daily Return of all Industries")
tight_layout()
#savefig("IndustrySortedStdDev.pdf")

subplot(212)
xlim(0, numberIndustries-1)
plot(100*IndustryReturnVector[sortedIndustries])
plot([0,numberIndustries-1],[0,0],"r--")
ylabel("Expected Daily Return (%)")
tight_layout()
savefig("images/IndustrySortedReturn.jpg")
;

### 1.C. Markets

In [ ]:
using PyPlot

# Obtain the sorted permutation of each market based on return

sortedMarkets = sortperm(MarketReturnVector)

# Plot the Market Standard Dev and Return in sorted order
fig = figure(figsize=(12,6))
subplot(211)
xlim(0, numberMarkets-1)
plot(100*sqrt.(MarketVarianceVector)[sortedMarkets], "g")
ylabel("Daily Standard Deviation (%)")
title("Daily Standard Deviation and Expected Daily Return of all Markets")
tight_layout()
#savefig("MarketSortedStdDev.pdf")

subplot(212)
xlim(0, numberIndustries-1)
plot(100*MarketReturnVector[sortedMarkets])
plot([0,numberMarkets-1],[0,0],"r--")
ylabel("Expected Daily Return (%)")
tight_layout()
#savefig("MarketSortedReturn.pdf")
;

## 2. Correlation Matrix

### 2.A. Securities

In [ ]:
using PyPlot, LinearAlgebra

# Plot the correlation matrix for securities
corr = diagm(0 => SecurityVarianceVector.^(-1/2))*SecurityCovarianceMatrix*diagm(0 => SecurityVarianceVector.^(-1/2))
imshow(corr[sortedSecurities,sortedSecurities]);colorbar();axis("image")
title("Correlation Matrix for all Securities")
tight_layout()
savefig("images/SecurityCorrelationMatrix.jpg")
;

### 2.B. Industries

In [ ]:
using PyPlot, LinearAlgebra

# Plot the correlation matrix for industries
corr = diagm(0 => IndustryVarianceVector.^(-1/2))*IndustryCovarianceMatrix*diagm(0 => IndustryVarianceVector.^(-1/2))
imshow(corr[sortedIndustries,sortedIndustries]);colorbar();axis("image")
title("Correlation Matrix for all Industries")
tight_layout()
savefig("images/IndustryCorrelationMatrix.jpg")

### 2.C. Markets

In [ ]:
using PyPlot, LinearAlgebra

# Plot the correlation matrix for markets
corr = diagm(0 => MarketVarianceVector.^(-1/2))*MarketCovarianceMatrix*diagm(0 => MarketVarianceVector.^(-1/2))
imshow(corr[sortedMarkets,sortedMarkets]);colorbar();axis("image")
title("Correlation Matrix for all Markets")
tight_layout()
#savefig("MarketCorrelationMatrix.pdf")

## 3. Standard Deviation vs Expected Daily Return Scatter Plot

### 3.A. Securities

In [ ]:
using PyPlot

# Plot the securities
figure(figsize=(7,5))
plot(100*sqrt.(SecurityVarianceVector), 100*SecurityReturnVector, "b.")
xlabel("Daily Standard Deviation (%)")
ylabel("Expected Daily Return (%)")
title("Security Daily Standard Deviation vs Expected Daily Return Scatter Plot")
tight_layout() 
savefig("images/SecurityReturnVsStdDevScatterPlot.jpg")
;

### 3.B. Industries

In [ ]:
using PyPlot

# Plot the industries
figure(figsize=(7,5))
plot(100*sqrt.(IndustryVarianceVector), 100*IndustryReturnVector, "b.")
xlabel("Daily Standard Deviation (%)")
ylabel("Expected Daily Return (%)")
title("Industry Daily Standard Deviation vs Expected Daily Return Scatter Plot")
tight_layout() 
savefig("images/IndustryReturnVsStdDevScatterPlot.jpg")
;

### 3.C. Markets

In [ ]:
using PyPlot

# Plot the markets
figure(figsize=(7,5))
plot(100*sqrt.(MarketVarianceVector), 100*MarketReturnVector, "b.")
xlabel("Daily Standard Deviation(%)")
ylabel("Expected Daily Return(%)")
title("Market Daily Standard Deviation vs Expected Daily Return Scatter Plot")
tight_layout() 
#savefig("MarketReturnVsStdDevScatterPlot.pdf")
;

## 4. Optimal Portfolio (λ = 1)

### 4.A. Securities

In [ ]:
using PyPlot

# Securities - λ = 1
figure(figsize=(12,8))
subplot(211)
xlim(0,numberSecurities)
bar(1:numberSecurities,100*x_optimal_securities_1[sortedSecurities])
title(string("Optimal Security Allocation for \$\\lambda=1\$"," ( \$\\mu=\$", 100*round(ret_securities_1,digits=3), "%, \$\\sigma=\$", 100*round(std_securities_1,digits=2), "% )"));
ylabel("Portfolio Allocation (%)")
tight_layout()
savefig("images/OptimalSecurityAllocation_1.jpg")
;

### 4.B. Industries

In [ ]:
using PyPlot

# Industries - λ = 1
figure(figsize=(12,8))
subplot(211)
xlim(0,numberIndustries)
bar(1:numberIndustries,100*x_optimal_industries_1[sortedIndustries])
title(string("Optimal Industry Allocation for \$\\lambda=1\$"," ( \$\\mu=\$", 100*round(ret_industries_1,digits=3), "%, \$\\sigma=\$", 100*round(std_industries_1,digits=2), "% )"));
ylabel("Portfolio Allocation (%)")
tight_layout()
savefig("images/OptimalIndustryAllocation_1.jpg")

### 4.C. Markets

In [ ]:
using PyPlot

# Markets - λ = 1
figure(figsize=(12,8))
subplot(211)
xlim(0,numberMarkets)
bar(1:numberMarkets,100*x_optimal_markets_1[sortedMarkets])
title(string("Optimal Market Allocation for \$\\lambda=1\$"," ( \$\\mu=\$", 100*round(ret_markets_1,digits=3), "%, \$\\sigma=\$", 100*round(std_markets_1,digits=2), "% )"));
ylabel("Portfolio Allocation (%)")
tight_layout()
#savefig("OptimalMarketAllocation_1.pdf")

## 5. Optimal Portfolio (λ = 3e-2)

### 5.A. Securities

In [ ]:
using PyPlot

# Securities - λ = 3e_2
figure(figsize=(12,8))
subplot(211)
xlim(0,numberSecurities)
bar(1:numberSecurities,100*x_optimal_securities_3e_2[sortedSecurities])
title(string("Optimal Security Allocation for \$\\lambda=3e_2\$"," ( \$\\mu=\$", 100*round(ret_securities_3e_2,digits=3), "%, \$\\sigma=\$", 100*round(std_securities_3e_2,digits=2), "% )"));
ylabel("Portfolio Allocation (%)")
tight_layout()
savefig("images/OptimalSecurityAllocation_3e_2.jpg")
;

### 5.B. Industries

In [ ]:
using PyPlot

# Industries - λ = 3e_2
figure(figsize=(12,8))
subplot(211)
xlim(0,numberIndustries)
bar(1:numberIndustries,100*x_optimal_industries_3e_2[sortedIndustries])
title(string("Optimal Industry Allocation for \$\\lambda=3e_2\$"," ( \$\\mu=\$", 100*round(ret_industries_3e_2,digits=3), "%, \$\\sigma=\$", 100*round(std_industries_3e_2,digits=2), "% )"));
ylabel("Portfolio Allocation (%)")
tight_layout()
savefig("images/OptimalIndustryAllocation_3e_2.jpg")

### 5.C. Markets

In [ ]:
using PyPlot

# Markets - λ = 3e_2
figure(figsize=(12,8))
subplot(211)
xlim(0,numberMarkets)
bar(1:numberMarkets,100*x_optimal_markets_3e_2[sortedMarkets])
title(string("Optimal Market Allocation for \$\\lambda=3e_2\$"," ( \$\\mu=\$", 100*round(ret_markets_3e_2,digits=3), "%, \$\\sigma=\$", 100*round(std_markets_3e_2,digits=2), "% )"));
ylabel("Portfolio Allocation (%)")
tight_layout()
#savefig("OptimalMarketAllocation_3e_2.pdf")

## 6. Paredo Curve

### 6.A. Securities

In [ ]:
using PyPlot

# Plot the optimal curve - Securities
plot(100*std_securities_paredo,100*ret_securities_paredo,"b-")
plot(100*sqrt.(diag(SecurityCovarianceMatrix)), 100*SecurityReturnVector, "k.", markersize=12)
# plot(sqrt.(diag(SecurityCovarianceMatrix))[interestingX_securities_1], SecurityReturnVector[interestingX_securities_1], "r.", markersize=12)  # low-risk portfolio
plot(100*sqrt.(diag(SecurityCovarianceMatrix))[interestingX_securities_3e_2], 100*SecurityReturnVector[interestingX_securities_3e_2], "r.", markersize=12)  # at the "elbow" of the curve
xlabel("Daily Standard Deviation - Close to Close (%)")
ylabel("Expected Daily Return - Close to Close (%)")
title("Paredo Curve - Securities")
axis([0,55,0,1.5]);
tight_layout()
savefig("images/SecurityParedoCurve.jpg")

### 6.B. Industries

In [ ]:
using PyPlot

# Plot the optimal curve - Industries
plot(100*std_industries_paredo,100*ret_industries_paredo,"b-")
plot(100*sqrt.(diag(IndustryCovarianceMatrix)), 100*IndustryReturnVector, "k.", markersize=12)
# plot(sqrt.(diag(IndustryCovarianceMatrix))[interestingX_industries_1], IndustryReturnVector[interestingX_industries_1], "r.", markersize=12)  # low-risk portfolio
plot(100*sqrt.(diag(IndustryCovarianceMatrix))[interestingX_industries_3e_2], 100*IndustryReturnVector[interestingX_industries_3e_2], "r.", markersize=12)  # at the "elbow" of the curve
xlabel("Daily Standard Deviation (%)")
ylabel("Expected Daily Return (%)")
title("Paredo Curve - Industries")
axis([0,55,0,1.5]);
tight_layout()
savefig("images/IndustryParedoCurve.jpg")

### 6.C. Markets

In [ ]:
using PyPlot

# Plot the optimal curve - Markets
plot(100*std_markets_paredo,100*ret_markets_paredo,"b-")
plot(100*sqrt.(diag(MarketCovarianceMatrix)), 100*MarketReturnVector, "k.", markersize=12)
# plot(sqrt.(diag(MarketCovarianceMatrix))[interestingX_markets_1], MarketReturnVector[interestingX_markets_1], "r.", markersize=12)  # low-risk portfolio
plot(100*sqrt.(diag(MarketCovarianceMatrix))[interestingX_markets_3e_2], 100*MarketReturnVector[interestingX_markets_3e_2], "r.", markersize=12)  # at the "elbow" of the curve
xlabel("Daily Standard Deviation (%)")
ylabel("Expected Daily Return (%)")
title("Paredo Curve - Markets")
axis([0,55,0,1.5]);
tight_layout()
#savefig("MarketParedoCurve.pdf")

### 6.D. Combined Optimization

In [ ]:
using PyPlot

# Plot the optimal curve 
plot(100*CombinedOptimizationStd,100*CombinedOptimizationReturn,"b-")
plot(100*sqrt.(diag(SecurityCovarianceMatrix)), 100*SecurityReturnVector, "k.", markersize=12)
plot(100*sqrt.(diag(SecurityCovarianceMatrix))[interestingCombinedSecurities], 100*SecurityReturnVector[interestingCombinedSecurities], "r.", markersize=12)  # at the "elbow" of the curve
xlabel("Daily Standard Deviation (%)")
ylabel("Expected Daily Return (%)")
title("Combined Optimal Portfolio - Paredo Curve")
axis([0,55,0,1.3]);
tight_layout()
savefig("images/CombinedOptimizationParedoCurve.jpg")

## 7. Optimal Combined Portfolio

In [ ]:
using PyPlot
figure(figsize=(12,8))

subplot(211)
xlim(0,numberSecurities)
bar(1:numberSecurities,100*optimalCombinedAllocationVector[sortedSecurities])
title(string("Optimal Combined Security Allocation for \$\\lambda=\$", highestReturnLambda, " ( \$\\mu=\$", 100*round(CombinedOptimizationReturn[highestReturnIndex],digits=3), "%, \$\\sigma=\$", 100*round(CombinedOptimizationStd[highestReturnIndex],digits=2), "% )"));
tight_layout()
savefig("images/OptimalCombinedSecurityAllocation.jpg")

## 8. Monte Carlo Simulation

### 8.1 Average Simulated 3-Month Loss

In [ ]:
using PyPlot

sortedLossValues = sortperm(MonteCarloSimulationAverageLosses)

fig = figure(figsize=(12,6))
subplot(212)
xlim(0, numberSecurities)
plot(100*MonteCarloSimulationAverageLosses[sortedLossValues])
plot([0,numberSecurities],[0,0],"r--")
ylabel("Monte Carlo Average Loss (%)")
title("Average Loss of Securities in Monte Carlo Simulations - Sorted on Loss %")
tight_layout()
savefig("images/MonteCarloAverageLoss.jpg")
;

### 8.2 Simulated 3-Month Loss Histograms

In [ ]:
using PyPlot

randomSecurityId = rand(1:length(securityKeys))
randomSecurityId2 = rand(1:length(securityKeys))
securityKey1 = securityIdMap[randomSecurityId]
securityKey2 = securityIdMap[randomSecurityId2]
Security1MonteCarloHistogramObject = MonteCarloHistogramCollection[securityKey1]
Security2MonteCarloHistogramObject = MonteCarloHistogramCollection[securityKey2]
security1SimulationHistogram = Security1MonteCarloHistogramObject.simulationHistogram
security1SimulationNumberDataPoints = Security1MonteCarloHistogramObject.numberDataPoints
security2SimulationHistogram = Security2MonteCarloHistogramObject.simulationHistogram
security2SimulationNumberDataPoints = Security2MonteCarloHistogramObject.numberDataPoints

s1i = findfirst(p -> p > 0, cumsum(security1SimulationHistogram))
e1i = findfirst(p -> p >= security1SimulationNumberDataPoints, cumsum(security1SimulationHistogram))

range1 = s1i:e1i

s2i = findfirst(p -> p > 0, cumsum(security2SimulationHistogram))
e2i = findfirst(p -> p >= security2SimulationNumberDataPoints, cumsum(security2SimulationHistogram))

range2 = s2i:e2i

figure(figsize=(12,8))
subplot(211)
xlim(HistogramBucketBoundaries[s1i,1],HistogramBucketBoundaries[e1i,2])
bar(HistogramBucketBoundaries[range1,3],security1SimulationHistogram[range1]./security1SimulationNumberDataPoints)
title(string("Monte Carlo Simulated Loss Histogram - ", securityKey1, " - (3 Months)"));
ylabel("Probability")
tight_layout()
#savefig("MonteCarloSimulatedLoss_1_close.pdf")

subplot(212)
xlim(HistogramBucketBoundaries[1,1],HistogramBucketBoundaries[end,2])
bar(HistogramBucketBoundaries[1:end,3],security1SimulationHistogram[1:end]./security1SimulationNumberDataPoints)
title(string("Monte Carlo Simuled Loss Histogram - ", securityKey1, " - (3 Months)"));
ylabel("Probability")
tight_layout()
savefig("images/MonteCarloSimulatedLoss_1_far.jpg")

figure(figsize=(12,8))
subplot(211)
xlim(HistogramBucketBoundaries[s2i,1],HistogramBucketBoundaries[e2i,2])
bar(HistogramBucketBoundaries[range2,3],security2SimulationHistogram[range2]./security2SimulationNumberDataPoints)
title(string("Monte Carlo Simuled Loss Histogram - ", securityKey2, " - (3 Months)"));
ylabel("Probability")
tight_layout()
#savefig("MonteCarloSimulatedLoss_2_close.pdf")

subplot(212)
xlim(HistogramBucketBoundaries[1,1],HistogramBucketBoundaries[end,2])
bar(HistogramBucketBoundaries[1:end,3],security2SimulationHistogram[1:end]./security2SimulationNumberDataPoints)
title(string("Monte Carlo Simuled Loss Histogram - ", securityKey2, " - (3 Months)"));
ylabel("Probability")
tight_layout()
savefig("images/MonteCarloSimulatedLoss_2_far.jpg")
;

### 8.3 99% Confidence Simulated Value at Risk and Conditional Value at Risk

In [ ]:
using PyPlot

sortedValueAtRiskIndices1 = sortperm(SecurityValueAtRisk[:,1])

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityValueAtRisk[sortedLossValues, 1], "g")
ylabel("Loss (%)")
title("Value At Risk (99% Confidence)")
tight_layout()
#savefig("SecuritySortedCloseStdDev.pdf")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityConditionalValueAtRisk[sortedLossValues, 1])
ylabel("Loss (%)")
title("Conditional Value At Risk (99% Confidence)")
tight_layout()
#savefig("SecuritySortedReturn.pdf")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*(SecurityConditionalValueAtRisk[sortedLossValues, 1] - SecurityValueAtRisk[sortedLossValues, 1]), "g")
ylabel("Loss (%)")
title("Difference of CoVAR and VAR (99% Confidence)")
tight_layout()

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityValueAtRisk[sortedValueAtRiskIndices1, 1], "g")
ylabel("Loss (%)")
title("Value At Risk (99% Confidence) - Sorted on VAR")
tight_layout()
#savefig("SecuritySortedCloseStdDev.pdf")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityConditionalValueAtRisk[sortedValueAtRiskIndices1, 1])
ylabel("Loss (%)")
title("Conditional Value At Risk (99% Confidence) - Sorted on VAR")
tight_layout()
#savefig("SecuritySortedReturn.pdf")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*(SecurityConditionalValueAtRisk[sortedValueAtRiskIndices1, 1] - SecurityValueAtRisk[sortedValueAtRiskIndices1, 1]), "g")
ylabel("Loss (%)")
title("Difference of CoVAR and VAR (99% Confidence) - Sorted on VAR")
tight_layout()

### 8.4 95% Confidence Simulated Value at Risk and Conditional Value at Risk

In [ ]:
using PyPlot

sortedValueAtRiskIndices2 = sortperm(SecurityValueAtRisk[:,2])

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityValueAtRisk[sortedLossValues, 2], "g")
ylabel("Loss (%)")
title("Security Value At Risk (95% Confidence)")
tight_layout()
savefig("images/security_var_95.jpg")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityConditionalValueAtRisk[sortedLossValues, 2])
ylabel("Loss (%)")
title("Security Conditional Value At Risk (95% Confidence)")
tight_layout()
savefig("images/security_cvar_95.jpg")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*(SecurityConditionalValueAtRisk[sortedLossValues, 2] - SecurityValueAtRisk[sortedLossValues, 2]), "g")
ylabel("Loss (%)")
title("Difference of CVaR and VaR (95% Confidence)")
tight_layout()
savefig("images/var_cvar_diff_95.jpg")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityValueAtRisk[sortedValueAtRiskIndices2, 2], "g")
ylabel("Loss (%)")
title("Security Value At Risk (95% Confidence) - Sorted on VaR")
tight_layout()
savefig("images/security_var_95_sort.jpg")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityConditionalValueAtRisk[sortedValueAtRiskIndices2, 2])
ylabel("Loss (%)")
title("Security Conditional Value At Risk (95% Confidence) - Sorted on VaR")
tight_layout()
savefig("images/security_cvar_95_sort.jpg")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*(SecurityConditionalValueAtRisk[sortedValueAtRiskIndices2, 2] - SecurityValueAtRisk[sortedValueAtRiskIndices2, 2]), "g")
ylabel("Loss (%)")
title("Difference of Security CVaR and VaR (95% Confidence) - Sorted on VaR")
tight_layout()
savefig("images/var_cvar_diff_95_sort.jpg")

### 8.5 90% Confidence Simulated Value at Risk and Conditional Value at Risk

In [ ]:
using PyPlot

sortedValueAtRiskIndices3 = sortperm(SecurityValueAtRisk[:,3])

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityValueAtRisk[sortedLossValues, 3], "g")
ylabel("Loss (%)")
title("Value At Risk (90% Confidence)")
tight_layout()
#savefig("SecuritySortedCloseStdDev.pdf")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityConditionalValueAtRisk[sortedLossValues, 3])
ylabel("Loss (%)")
title("Conditional Value At Risk (90% Confidence)")
tight_layout()
#savefig("SecuritySortedReturn.pdf")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*(SecurityConditionalValueAtRisk[sortedLossValues, 3] - SecurityValueAtRisk[sortedLossValues, 3]), "g")
ylabel("Loss (%)")
title("Difference of CoVAR and VAR (90% Confidence)")
tight_layout()

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityValueAtRisk[sortedValueAtRiskIndices3, 3], "g")
ylabel("Loss (%)")
title("Value At Risk (90% Confidence) - Sorted on VAR")
tight_layout()
#savefig("SecuritySortedCloseStdDev.pdf")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*SecurityConditionalValueAtRisk[sortedValueAtRiskIndices3, 3])
ylabel("Loss (%)")
title("Conditional Value At Risk (90% Confidence) - Sorted on VAR")
tight_layout()
#savefig("SecuritySortedReturn.pdf")

fig = figure(figsize=(12,8))
subplot(211)
xlim(0, numberSecurities)
plot(100*(SecurityConditionalValueAtRisk[sortedValueAtRiskIndices3, 3] - SecurityValueAtRisk[sortedValueAtRiskIndices3, 3]), "g")
ylabel("Loss (%)")
title("Difference of CoVAR and VAR (90% Confidence) - Sorted on VAR")
tight_layout()

### 8.6 Time Series Daily Periodic Return Metrics

In [ ]:
using PyPlot

# Plot the Security Standard Dev and Return in sorted order
fig = figure(figsize=(12,6))
subplot(211)
xlim(0, numberSecurities)
plot(100*PeriodicReturnStdDev[sortedLossValues], "g")
ylabel("Standard Deviation (%)")
title("Security Daily Periodic Loss and Standrad Deviation")
tight_layout()
#savefig("SecuritySortedCloseStdDev.pdf")

subplot(212)
xlim(0, numberSecurities)
plot(-100*PeriodicReturnAverage[sortedLossValues])
plot([0,numberSecurities],[0,0],"r--")
ylabel("Average Periodic Loss (%)")
tight_layout()
savefig("images/SecurityPeriodicLoss.jpg")

fig = figure(figsize=(12,6))
subplot(211)
xlim(0, numberSecurities)
plot(100*PeriodicReturnDrift[sortedLossValues])
plot([0,numberSecurities],[0,0],"r--")
title("Security Drift")
ylabel("Drift (%)")
tight_layout()
savefig("images/SecurityDrift.jpg")
;